## <font color='289C4E' font-size='100px'>Table of contents<font><a class='anchor' id='top'></a>
1. [Load in Data](#Load-in-Data)
- Error Checking
- Add Columns
- [Output ShotCSV](#Output-ShotCSV)
2. [Create Point DF](#Create-Point-DF)
- Add Columns
- [Output PointCSV for Visuals](#PRINT-POINTCSV-VISUALS)
- [Output PointCSV for Upload](#Print-POINTCSV-UPLOAD)    
        - cut out points with no timestamp position    
        - atNetPlayer1 has values of the player name for display purposes instead of boolean values

[Click for Summary Stats and EDA HERE](#EDA)

## <font color='289C4E' font-size='100px'>What Requires Inputs<font><a class='anchor' id='top'></a>
1. [Fill in File Input](#Load-in-Data) => <font color='8B0000'>Mandatory<font>
2. [Output ShotCSV](#Output-ShotCSV) => <font color='BBED99'>Optional<font>
3. [EDA and get Stats Here](#EDA) => <font color='BBED99'>Optional<font>
4. [Output PointCSV for Visuals](#PRINT-POINTCSV-VISUALS) => <font color='BBED99'>Optional<font>
5. [Output PointCSV for Upload](#Print-POINTCSV-UPLOAD) => <font color='BBED99'>Optional<font>

## <font color='289C4E' font-size='100px'>Coloring Info<font><a class='anchor' id='top'></a>
1. set_background('#90EE90') => For check passing
2. set_background('#FF474C') => For check failing
3. set_background('#E9FDFF') => For cell that require inputs + are used for cleaning data. 

## <font color='289C4E' font-size='100px'>TODO<font><a class='anchor' id='top'></a>

###  Cameryn 
[Depth Count for each Point](#Add-Columns:-Depths-Count-(Short,-Deep))

[EDA for Depth of GroundStrokes](#EDA)

### Angelo 
[Change atNetPlayer to be aggregated from Coordinate Data](#Add-Column:-atNetPlayer1,-atNetPlayer2)
- atNetPlayer1 and atNetPlayer2 change to be aggreated from x and y coordinates
- Look into if isVolley column can be determined if the shotLocation and shotContact are close too each other

### (Insert Name)
[Change isApproach to be aggregatead from Coordinate Data](####-Add-Column:-isAprroach)
- see if the next consecutive shotInRally coordinates are further up into the court


### Add Error Checks
#### Add Error check to output rows where firstServeIn == NA and secondServeIn == NA AND is not a Double Falue

# Setup

## Style

In [65]:
#!pip install jupyterthemes
#from jupyterthemes import get_themes
#import jupyterthemes as jt
#from jupyterthemes.stylefx import set_nb_theme
#set_nb_theme('monokai')

In [360]:
from IPython.core.magic import register_cell_magic
from IPython.display import HTML, display

@register_cell_magic
def set_background(color, cell=None):
    script = (
        "var cell = this.closest('.jp-CodeCell');"
        "var editor = cell.querySelector('.jp-Editor');"
        "editor.style.background='{}';"
        "this.parentNode.removeChild(this)"
    ).format(color)

    display(HTML('<img src onerror="{}">'.format(script)))

## Reorder csv

In [94]:
import pandas as pd
import numpy as np
set_background('#E9FDFF')

In [71]:
your_file_name = '(INSERT FILE NAME HERE)'
shot_data = pd.read_csv(your_file_name)
set_background('#GGGGGG')

FileNotFoundError: [Errno 2] No such file or directory: '(INSERT FILE NAME HERE)'

In [ ]:
# Check existing columns
shot_data.columns

In [ ]:
desired_order = ['pointScore', 'gameScore', 'setScore', 'isPointStart', 'pointStartTime',
       'isPointEnd', 'pointEndTime', 'pointNumber', 'isBreakPoint',
       'shotInRally', 'side', 'serverName', 'serverFarNear', 'firstServeIn',
       'firstServeZone', 'firstServeXCoord', 'firstServeYCoord',
       'secondServeIn', 'secondServeZone', 'secondServeXCoord',
       'secondServeYCoord', 'isAce', 'shotContactX', 'shotContactY',
       'shotDirection', 'shotFhBh', 'isSlice', 'isVolley', 'isOverhead',
       'isApproach', 'isDropshot', 'isExcitingPoint', 'atNetPlayer1',
       'atNetPlayer2', 'isLob', 'shotLocationX', 'shotLocationY', 'isWinner',
       'isErrorWideR', 'isErrorWideL', 'isErrorNet', 'isErrorLong',
       'clientTeam', 'Date', 'Division', 'Event', 'lineupPosition',
       'matchDetails', 'matchVenue', 'opponentTeam', 'player1Name',
       'player2Name', 'player1Hand', 'player2Hand', 'Round', 'Surface',
       'Notes']

shot_data = shot_data.reindex(columns=desired_order)

In [ ]:
# Save point_df to CSV file

# Assuming point_df is your DataFrame and player1Name and player2Name are the names from the first row
player1NameNoSpace = shot_data.iloc[0]['player1Name'].replace(" ", "")
player2NameNoSpace = shot_data.iloc[0]['player2Name'].replace(" ", "")

# Save DataFrame to CSV file with modified player names
shot_data.to_csv(f'(Cleaning) {player1NameNoSpace}_{player2NameNoSpace}.csv', index=False)


## Load in Data

In [207]:
import pandas as pd
import numpy as np
import re

In [129]:
# Put your Uncleaned .csv file name here
your_file_name ='(Cleaned)Rudy Quan vs Yuta Shimizu Little Rock.csv'
shot_data = pd.read_csv(your_file_name)
set_background('#E9FDFF')

In [105]:
# Check existing columns
shot_data.columns

Index(['pointScore', 'gameScore', 'setScore', 'isPointStart', 'pointStartTime',
       'isPointEnd', 'pointEndTime', 'pointNumber', 'isBreakPoint',
       'shotInRally', 'side', 'serverName', 'serverFarNear', 'firstServeIn',
       'firstServeZone', 'firstServeXCoord', 'firstServeYCoord',
       'secondServeIn', 'secondServeZone', 'secondServeXCoord',
       'secondServeYCoord', 'isAce', 'shotContactX', 'shotContactY',
       'shotDirection', 'shotFhBh', 'isSlice', 'isVolley', 'isOverhead',
       'isApproach', 'isDropshot', 'isExcitingPoint', 'atNetPlayer1',
       'atNetPlayer2', 'isLob', 'shotLocationX', 'shotLocationY', 'isWinner',
       'isErrorWideR', 'isErrorWideL', 'isErrorNet', 'isErrorLong',
       'clientTeam', 'Date', 'Division', 'Event', 'lineupPosition',
       'matchDetails', 'matchVenue', 'opponentTeam', 'player1Name',
       'player2Name', 'player1Hand', 'player2Hand', 'Round', 'Surface',
       'Notes'],
      dtype='object')

In [107]:
desired_order = ['pointScore', 'gameScore', 'setScore', 'isPointStart', 'pointStartTime',
       'isPointEnd', 'pointEndTime', 'pointNumber', 'isBreakPoint',
       'shotInRally', 'side', 'serverName', 'serverFarNear', 'firstServeIn',
       'firstServeZone', 'firstServeXCoord', 'firstServeYCoord',
       'secondServeIn', 'secondServeZone', 'secondServeXCoord',
       'secondServeYCoord', 'isAce', 'shotContactX', 'shotContactY',
       'shotDirection', 'shotFhBh', 'isSlice', 'isVolley', 'isOverhead',
       'isApproach', 'isDropshot', 'isExcitingPoint', 'atNetPlayer1',
       'atNetPlayer2', 'isLob', 'shotLocationX', 'shotLocationY', 'isWinner',
       'isErrorWideR', 'isErrorWideL', 'isErrorNet', 'isErrorLong',
       'clientTeam', 'Date', 'Division', 'Event', 'lineupPosition',
       'matchDetails', 'matchVenue', 'opponentTeam', 'player1Name',
       'player2Name', 'player1Hand', 'player2Hand', 'Round', 'Surface',
       'Notes']

shot_data = shot_data.reindex(columns=desired_order)

### Manually fill in meta data

In [110]:
# Fill in meta data
shot_data['clientTeam'] = 'UCLA'
shot_data['Date'] = '5/31/24'
shot_data['Division'] = ''
shot_data['Event'] = 'Little Rock Challenger'
shot_data['lineupPosition'] = ''
shot_data['matchDetails'] = ''
shot_data['matchVenue'] = ''
shot_data['opponentTeam'] = ''
shot_data['player1Name'] = 'Rudy Quan'
shot_data['player2Name'] = 'Yuta Shimizu'
shot_data['player1Hand'] = 'Right'
shot_data['player2Hand'] = 'Left'
shot_data['Round'] = 'Quarter-final'
shot_data['Surface'] = 'Hard'
shot_data['Notes'] = ''

### Assign Player Names

In [113]:
player1_new_name = shot_data['player1Name'].iloc[0]
player2_new_name = shot_data['player2Name'].iloc[0]

# Replace values in the serverName column
shot_data['serverName'] = shot_data['serverName'].replace({'Player1': player1_new_name, 'Player2': player2_new_name})

In [115]:
shot_data['serverName'].unique()

array(['Rudy Quan', 'Yuta Shimizu'], dtype=object)

# Notebook Start

## Error Check 1:
#### Check that Player1Name and Player2Name have values

In [139]:
# fillInPlayer1Name = "Choose_Player1Name_Here"
# fillInPlayer2Name = "Choose_Player2Name_Here"

if shot_data.at[0, 'player1Name'] == None:
    print("player1Name was blank. Filling with 'fillInplayer1Name'")
    shot_data.at[0, 'player1Name'] = fillInPlayer1Name
else:
    print("Check Passed: player1Name is not blank. Current value:", shot_data.at[0, 'player1Name'])

# Check if player2Name is blank, if so, fill it with 'fillInplayer2Name'
if shot_data.at[0, 'player2Name'] == None:
    print("player2Name was blank. Filling with 'fillInplayer2Name'")
    shot_data.at[0, 'player2Name'] = fillInPlayer2Name
    set_background('#FF474C')
else:
    print("Check Passed: player2Name is not blank. Current value:", shot_data.at[0, 'player2Name'])
    set_background('#90EE90')

Check Passed: player1Name is not blank. Current value: nan
Check Passed: player2Name is not blank. Current value: nan


## Error Check 2:
#### Check if there are missing 'Deuce' or 'Ad' sides in side column

In [150]:
# Filter out rows with 'side' values other than 'Deuce' and 'Ad'
filtered_data = shot_data[shot_data['side'].isin(['Deuce', 'Ad'])]

# Rows that don't meet the condition
rows_not_meeting_condition = shot_data[~shot_data['side'].isin(['Deuce', 'Ad'])]

# Count of rows not meeting the condition
count_rows_not_meeting_condition = len(rows_not_meeting_condition)
if count_rows_not_meeting_condition == 0:
    print("Check Passed. Missing #: {}".format(count_rows_not_meeting_condition))
    set_background('#90EE90')
else:
    print("Rows with Side that aren't Deuce or Ad:", count_rows_not_meeting_condition)
    
    # Output rows if you want to see which rows
    print("Rows not meeting the condition:")
    print(rows_not_meeting_condition[['pointScore', 'gameScore', 'setScore','side', 'shotInRally']])
    set_background('#FF474C')

Check Passed. Missing #: 0


## Error Check 3: 
#### Fills in PointNumber if not there

In [156]:
 # Check if both conditions passed
if 'pointNumber' in shot_data.columns and not shot_data['pointNumber'].isnull().any() and shot_data['pointNumber'].is_monotonic_increasing:
    print("Check Passed: Point Numbers already exist")


# Check if pointNumber exists for every row and is not empty
if 'pointNumber' not in shot_data.columns or shot_data['pointNumber'].isnull().any():
    # Execute the script to calculate pointNumber
    point_starts = (shot_data['isPointStart'] == 1)
    shot_data['pointNumber'] = point_starts.cumsum()
    
    print("Data had missing point numbers. They were filled in automatically")
    
# Check if pointNumber is in increasing order
if not shot_data['pointNumber'].is_monotonic_increasing:
    # Print the condition failure if the 'pointNumber' column is not in increasing order
    print("Data had point numbers in the wrong order. They were filled in automatically")
    point_starts = (shot_data['isPointStart'] == 1)
    shot_data['pointNumber'] = point_starts.cumsum()
set_background('#90EE90')

Check Passed: Point Numbers already exist


In [158]:
missing_pointNumber_rows = shot_data[shot_data['pointNumber'].isnull()]

# Check if there are missing rows
if len(missing_pointNumber_rows) == 0:
    print("Check Passed: All rows have pointNumber")
    set_background('#90EE90')
else:
    print(f"Count of rows missing 'pointNumber': {len(missing_pointNumber_rows)}")
    set_background('#FF474C')

Check Passed: All rows have pointNumber


## Add Column:
#### player1Hand, player2Hand

In [162]:
# fillInplayer1Hand = "Right"
# fillInplayer2Hand = "Right"

# Check if player1Hand key exists in shot_data, if not, create it
if 'player1Hand' not in shot_data:
    shot_data['player1Hand'] = None

# Check if player2Hand key exists in shot_data, if not, create it
if 'player2Hand' not in shot_data:
    shot_data['player2Hand'] = None

if shot_data.at[0, 'player1Hand'] == None:
    print("player1Hand was blank. Filling with 'fillInplayer1Hand'")
    shot_data.at[0, 'player1Hand'] = fillInplayer1Hand
else:
    print("Check Passed: player1Hand is not blank. Current value:", shot_data.at[0, 'player1Hand'])

# Check if player2Hand is blank, if so, fill it with 'fillInplayer2Hand'
if shot_data.at[0, 'player2Hand'] == None:
    print("player2Hand was blank. Filling with 'fillInplayer2Hand'")
    shot_data.at[0, 'player2Hand'] = fillInplayer2Hand
else:
    print("Check Passed: player2Hand is not blank. Current value:", shot_data.at[0, 'player2Hand'])


print("Player1Hand Current value:", shot_data.at[0, 'player1Hand'])
print("Player2Hand Current value:", shot_data.at[0, 'player2Hand'])
set_background('#90EE90')

Check Passed: player1Hand is not blank. Current value: nan
Check Passed: player2Hand is not blank. Current value: nan
Player1Hand Current value: nan
Player2Hand Current value: nan


# Forward Fill GameScore, SetScore, tiebreakScore, serverName, player1Name, player2Name

### Filling Columns: Forward Fill
- 'gameScore'
- 'setScore'
- 'serverName'
- 'player1Name',
- 'player2Name'
- 'player2Hand'

In [167]:
columns_to_fill = ['gameScore','setScore', 'serverName', 'player1Name', 'player2Name', 'player1Hand', 'player2Hand'] 

for column in columns_to_fill:
    shot_data[column] = shot_data[column].replace(['', 'na'], pd.NaT)
    shot_data[column] = shot_data[column].ffill()
set_background('#90EE90')

## Error Check 4: 
#### Check for NA's

- all 0, except missing pointScore should match amount of tiebreak shots
- side can have missing, for old tagger and not tagging side of each shot

### Fixing:
- Open CSV in __Google Sheets__ (Excel will change to date format), output game, set, point Score of missing values. Then find in google sheets, adjust, redownload, and re-upload into notebook

In [175]:
# Count empty strings in each column
empty_string_counts = (shot_data == "").sum()

# Filter out columns with zero empty strings
non_zero_counts = empty_string_counts[empty_string_counts > 0]

# Count NaN values in 'pointScore' column when 'gameScore' is '6-6'
na_tiebreak_count = shot_data.loc[shot_data['gameScore'] == '6-6', 'pointScore'].isna().sum()

side_na_count = shot_data.loc[shot_data['isPointStart'] == 1, 'side'].isna().sum()
print("\nCount of NaN values for 'side' on Point Start:", side_na_count)

# Display the count
# print("\nCount of Nan when gameScore is '6-6' (# tiebreak shots):", na_tiebreak_count)


# Count NaN or empty values in specified columns
na_counts = shot_data[[ 'pointScore', 'shotInRally', 'gameScore', 'setScore', 'side', 'serverName']].isna().sum()

# Display the counts
print(f"\nCount of NA's in these columns\n")
print(na_counts)

# Display the counts
print("\nCount of empty strings in each column that includes at least one:\n")
print(non_zero_counts)

if sum(na_counts) == 0 and len(non_zero_counts) == 0:
    set_background('#90EE90')
else:
    set_background('#FF474C')



Count of NaN values for 'side' on Point Start: 0

Count of NA's in these columns

pointScore     0
shotInRally    0
gameScore      0
setScore       0
side           0
serverName     0
dtype: int64

Count of empty strings in each column that includes at least one:

Series([], dtype: int64)


## Error Check 9: 
####  Outputs Missing shotInRally Rows

In [180]:
empty_shot_rows = shot_data[shot_data['shotInRally'].isnull()]

if not empty_shot_rows.empty:
    # Iterate over the index of empty_shot_rows
    for index in empty_shot_rows.index:
        # Get the row with empty shotInRally
        empty_row = shot_data.loc[index]
        
        # Get the row above it
        if index - 1 >= 0:
            above_row = shot_data.loc[index - 1]
        else:
            above_row = None
        
        # Get the row below it
        if index + 1 < len(shot_data):
            below_row = shot_data.loc[index + 1]
        else:
            below_row = None
        
        # Print the rows
        print("Empty row:")
        print(empty_row)
        print("Row above:")
        print(above_row)
        print("Row below:")
        print(below_row)
        print("====================")
        set_background('#FF474C')
else:
    print("Check Passed: No Missing shotInRally")
    set_background('#90EE90')


Check Passed: No Missing shotInRally


## Error Check 5: 
####  Check that pointNumber is increasing consecutively

- if video is bad and points are cut out, this can fail and that's ok.

In [185]:
point_numbers = shot_data['pointNumber'].unique()

# Check if the point numbers are consecutive
if list(point_numbers) == list(range(1, len(point_numbers) + 1)):
    print("Check Passed")
    set_background('#90EE90')
else:
    raise ValueError("Error: The 'pointNumber' column does not contain consecutive numbers starting from 1.")
    set_background('#FF474C')


Check Passed


## Error Check 6: 
#### Check same amount of start and end points

In [190]:
# Count of Point Start and Point End
num_point_start = shot_data['isPointStart'].sum()

# Count the number of rows where isPointEnd is equal to 1
num_point_end = shot_data['isPointEnd'].sum()

if num_point_start != num_point_end:
    print("Number of rows with isPointStart = 1:", num_point_start)
    print("Number of rows with isPointEnd = 1:", num_point_end)
    raise ValueError("Error: count of isPointStart = 1 and isPointEnd = 1 are not the same.")
    set_background('#FF474C')
else:
    print("Number of rows with isPointStart = 1:", num_point_start)
    print("Number of rows with isPointEnd = 1:", num_point_end)
    print('Check Passed.')
    set_background('#90EE90')

Number of rows with isPointStart = 1: 135.0
Number of rows with isPointEnd = 1: 135.0
Check Passed.


## Error Check 7: 
#### Output consecutive isPointStart with no isPointEnd, and vice versa for isPointEnd


In [195]:
# Shubhan

# Output total start, total end
# How many Points aren't enclosing between point start and point end 

# Count where isPointStart = 1
count_isPointStart = (shot_data['isPointStart'] == 1).sum()

# Count where isPointEnd = 1
count_isPointEnd = (shot_data['isPointEnd'] == 1).sum()

print("Total count where isPointStart = 1:", count_isPointStart)
print("Total count where isPointEnd = 1:", count_isPointEnd)

# Output Point/Game Scores where there is consecutive isPointStart with no isPointEnd
def check_isPointStart_in_between(shot_data):
    start_indices = shot_data.index[shot_data['isPointStart'] == 1]  # Get indices where isPointStart is True
    end_indices = shot_data.index[shot_data['isPointEnd'] == 1]  # Get indices where isPointEnd is True
    start_without_end = []

    for i in range(len(start_indices) - 1):
        current_start_index = start_indices[i]
        next_start_index = start_indices[i + 1]

        # Get all isPointEnd indices between current_start_index and next_start_index
        end_between_starts = end_indices[(end_indices >= current_start_index) & (end_indices < next_start_index)]

        # Check if there is at least one isPointEnd between current_start_index and next_start_index
        if len(end_between_starts) == 0:
            start_without_end.append(current_start_index)

    # Check the last isPointStart
    last_start_index = start_indices[-1]
    end_after_last_start = end_indices[end_indices >= last_start_index]

    if len(end_after_last_start) == 0:
        start_without_end.append(last_start_index)

    if start_without_end:
        print(" ")
        print("isPointStart without corresponding isPointEnd:")
        for index in start_without_end:
#             print("Index:", index)
#             print("pointScore:", shot_data.at[index, 'pointScore'])
#             print("gameScore:", shot_data.at[index, 'gameScore'])
#             print("setScore:", shot_data.at[index, 'setScore'])
            print(" ")
            print(shot_data.loc[start_without_end][['pointScore', 'gameScore', 'setScore', 'isPointStart']])
            print(" ")

        set_background('#FF474C')
        return False
    else:
        print("All isPointStart have corresponding isPointEnd.")
        set_background('#90EE90')
        return True

# Check if the condition is met
check_isPointStart_in_between(shot_data)


# Output Point/Game Scores where there is consecutive isPointEnd with no isPointStart
def check_isPointEnd_in_between(shot_data):
    start_indices = shot_data.index[shot_data['isPointStart'] == 1]  # Get indices where isPointStart is True
    end_indices = shot_data.index[shot_data['isPointEnd'] == 1]  # Get indices where isPointEnd is True
    end_without_start = []

    for i in range(len(end_indices) - 1):
        current_end_index = end_indices[i]
        next_end_index = end_indices[i + 1]

        # Get all isPointStart indices b/w current_end_index and next_end_index
        start_between_ends = start_indices[(start_indices > current_end_index) & (start_indices <= next_end_index)]

        # Check if there is at least one isPointStart between current_end_index and next_end_index
        if len(start_between_ends) == 0:
            end_without_start.append(current_end_index)

    # NOTE: This will always append the last index of the sheet because the last end will never have a corresponding start so I'm not sure if its necessary to add it to the end_without_start array
    # Check the last isPointStart
    """
    last_end_index = end_indices[-1]
    start_after_last_end = start_indices[start_indices >= last_end_index]

    if len(start_after_last_end) == 0:
        end_without_start.append(last_end_index)
    """

    if end_without_start:
        print("isPointEnd without corresponding isPointStart:")
#         for index in end_without_start:
#             print("Index:", index)
#             print("pointScore:", shot_data.at[index, 'pointScore'])
#             print("gameScore:", shot_data.at[index, 'gameScore'])
#             print("setScore:", shot_data.at[index, 'setScore'])
#             print("setScore:", shot_data.at[index, 'tiebreakScore'])

        # Output the rows of shot_data that correspond to the indices in end_without_start
        print(" ")
        print(shot_data.loc[end_without_start][['pointScore', 'gameScore', 'setScore', 'isPointEnd']])
        set_background('#FF474C')
        return False
    else:
        print("All isPointEnd have corresponding isPointStart.")
        set_background('#90EE90')
        return True

# Check if the condition is met
check_isPointEnd_in_between(shot_data)

Total count where isPointStart = 1: 135
Total count where isPointEnd = 1: 135
All isPointStart have corresponding isPointEnd.


All isPointEnd have corresponding isPointStart.


True

## Error Check 8:
#### Change the score from incorrect date format
- accounts for all variations of dates eg. (0-00-0000, 0/0/0000)

In [198]:
## Error Check 13:
#### Make Jan-00 back into 1-0 for Game/Set Score

# Make Scores Strings not Date Time
columns_to_convert = ['gameScore', 'setScore'] #if no tiebreakScore
# columns_to_convert = ['gameScore', 'setScore', 'tiebreakScore']
shot_data[columns_to_convert] = shot_data[columns_to_convert].astype(object)

In [202]:
import re

# Define a mapping for month abbreviations
month_mapping = {'Jan': '1', 'Feb': '2', 'Mar': '3', 'Apr': '4', 'May': '5', 'Jun': '6',
                 'Jul': '7', 'Aug': '8', 'Sep': '9', 'Oct': '10', 'Nov': '11', 'Dec': '12'}

# Function to convert string like 'Jan-00' to '1-0'
def convert_score_string(score_str):
    # Check if the string has a month abbreviation and a year ending with '00'
    if re.match(r'^\d{1,2}-[A-Za-z]{3}$', score_str):
        # Extract year and month abbreviation
        year, month = score_str.split('-')

        # Remove leading zeros from the year
        year = str(int(year))

        # Replace month abbreviation with corresponding number
        month_number = month_mapping.get(month, month)

        # Concatenate the parts to form the transformed string
        transformed_str = f'{year}-{month_number}'
        return transformed_str

    # Check if the string has a month abbreviation and a year with leading '0's
    elif re.match(r'^[A-Za-z]{3}-\d{1,2}$', score_str):
        # Extract month abbreviation and year
        month, year = score_str.split('-')

        # Replace month abbreviation with corresponding number
        month_number = month_mapping.get(month, month)

        # Remove leading zeros from the year
        year = str(int(year))

        # Concatenate the parts to form the transformed string
        transformed_str = f'{month_number}-{year}'
        return transformed_str

    # Check if the string has a date in the format 'month/day/year'
    elif re.match(r'^\d{1,2}/\d{1,2}/\d{4}$', score_str):
        # Extract month, day, and year
        month, day, year = score_str.split('/')

        # Remove leading zeros from month and day
        month = str(int(month))
        day = str(int(day))

        # Concatenate the parts to form the transformed string
        transformed_str = f'{month}-{day}'
        return transformed_str

    # Check if the string has a date in the format 'month-day-year'
    elif re.match(r'^\d{1,2}-\d{1,2}-\d{4}$', score_str):
        # Extract month, day, and year
        month, day, year = score_str.split('-')

        # Remove leading zeros from month and day
        month = str(int(month))
        day = str(int(day))

        # Concatenate the parts to form the transformed string
        transformed_str = f'{month}-{day}'
        return transformed_str

    return score_str

# Apply the conversion function to the relevant columns in shot_data
shot_data['gameScore'] = shot_data['gameScore'].apply(convert_score_string)
shot_data['setScore'] = shot_data['setScore'].apply(convert_score_string)
# shot_data['tiebreakerScore'] = shot_data['tiebreakerScore'].apply(convert_score_string)
set_background('#90EE90')

## Error Check 9: 
#### Check that there is no strings in set, game scores. 

In [213]:
# Assuming shot_data is your DataFrame
unique_set_scores = shot_data['setScore'].unique()
unique_game_scores = shot_data['gameScore'].unique()

# Print unique values
print("Unique Set Scores:", unique_set_scores)
print("Unique Game Scores:", unique_game_scores)

set = [0,1,2]
game = [0,1,2,3,4,5,6]

for j in unique_set_scores:
    b = [int(s) for s in re.findall(r'\b\d+\b', j)]
    for t in b:
        if t not in set:
            set_background('#FF474C')
            
for j in unique_game_scores:
    b = [int(s) for s in re.findall(r'\b\d+\b', j)]
    for t in b:
        if t not in game:
            set_background('#FF474C')
set_background('#90EE90')

Unique Set Scores: ['0-0' '0-1' '1-1']
Unique Game Scores: ['0-0' '1-0' '2-0' '2-1' '2-2' '2-3' '2-4' '3-4' '4-4' '4-5' '0-1' '1-1'
 '3-1' '3-2' '3-3' '3-5']


## Error Check 10 : 
### Print out all the rows where isPointStart time doesn't start in the beginning of a rally

In [215]:
# Filter the DataFrame
filtered_rows = shot_data[(shot_data['isPointStart'] == 1) & (shot_data['shotInRally'] != 1)]

# Print the filtered rows
if filtered_rows.empty:
    print ("Check passed.")
    set_background('#90EE90')
else:
    print("Rows where isPointStart = 1 and shotInRally != 1:")
    print(filtered_rows)
    set_background('#FF474C')


Check passed.


## Error Check 11: 
### Find the rows where isPointEnd = 1 and shotInRally = 1 but is neither an ace or a double fault

In [217]:
# Filter the DataFrame
filtered_rows = shot_data[
    (shot_data['isPointEnd'] == 1) & 
    (shot_data['shotInRally'] == 1) & 
    (shot_data['firstServeIn'] != 0) &
    (shot_data['secondServeIn'] != 0) &
    (shot_data['isAce'] != 1)
]

# Print the filtered rows
if filtered_rows.empty:
    print ("Check passed.")
    set_background('#90EE90')
else:
    print("Rows where isPointEnd = 1, shotInRally = 1, firstServeIn and secondServeIn are both not 0, and isAce != 1:")
    print(filtered_rows)
    set_background('#FF474C')

Check passed.


## Error Check 12: 
### Find all the rows where there are duplicate isPointStarts == 1 points
- Finds all the points where the there are the same score and isPointStart == 1
#### Warning: This will output rows that have pointScore such as "40-40", "40-A", "A-40"

In [231]:
# Filter the DataFrame
filtered_rows = shot_data[shot_data['isPointStart'] == 1]

# Output rows where isPointStart is 1 and pointScore, gameScore, and setScore have the same value
output_rows = filtered_rows[filtered_rows.duplicated(subset=['pointScore', 'gameScore', 'setScore'], keep=False)]

# Print the output rows
if output_rows.empty:
    print ("Check passed.")
    set_background('#90EE90')
else:
    print("Rows where isPointStart is 1 and pointScore, gameScore, and setScore have the same value:")
    print(output_rows[['pointScore', 'gameScore', 'setScore', 'isPointStart']])
    wrong = False
    for j in output_rows['pointScore']:
        b = j.split('-')
        for l in b:
            if l not in ['40','A']:
                wrong = True
                break
    if wrong:
        set_background('#FF474C')
    else:
        set_background('#90EE90')
    



Rows where isPointStart is 1 and pointScore, gameScore, and setScore have the same value:
    pointScore gameScore setScore  isPointStart
42       40-40       0-0      0-0           1.0
53       40-40       0-0      0-0           1.0
63        A-40       0-0      0-0           1.0
68       40-40       0-0      0-0           1.0
74        A-40       0-0      0-0           1.0
79       40-40       0-0      0-0           1.0
83        A-40       0-0      0-0           1.0
88       40-40       0-0      0-0           1.0
90        A-40       0-0      0-0           1.0
95       40-40       0-0      0-0           1.0
101       A-40       0-0      0-0           1.0
104      40-40       0-0      0-0           1.0
106       A-40       0-0      0-0           1.0
472      40-40       4-5      0-0           1.0
482      40-40       4-5      0-0           1.0
611       40-A       1-1      0-1           1.0
618      40-40       1-1      0-1           1.0
620       A-40       1-1      0-1           1.

## Error Check 13: 
### Find all the rows where there are duplicate isPointEnd == 1 points
- Finds all the points where the there are the same score and isPointEnd == 1

In [239]:
# Filter the DataFrame
filtered_rows = shot_data[shot_data['isPointEnd'] == 1]

# Output rows where isPointStart is 1 and pointScore, gameScore, and setScore have the same value
output_rows = filtered_rows[filtered_rows.duplicated(subset=['pointScore', 'gameScore', 'setScore'], keep=False)]

# Print the output rows
if output_rows.empty:
    print ("Check passed.")
    set_background('#90EE90')
else:
    print("Rows where isPointEnd is 1 and pointScore, gameScore, and setScore have the same value:")
    print(output_rows[['pointScore', 'gameScore', 'setScore', 'isPointStart']])
    wrong = False
    for j in output_rows['pointScore']:
        b = j.split('-')
        for l in b:
            if l not in ['40','A']:
                wrong = True
                break
    if wrong:
        set_background('#FF474C')
    else:
        set_background('#90EE90')


Rows where isPointEnd is 1 and pointScore, gameScore, and setScore have the same value:
    pointScore gameScore setScore  isPointStart
48       40-40       0-0      0-0           NaN
62       40-40       0-0      0-0           NaN
67        A-40       0-0      0-0           NaN
73       40-40       0-0      0-0           NaN
78        A-40       0-0      0-0           NaN
82       40-40       0-0      0-0           NaN
87        A-40       0-0      0-0           NaN
89       40-40       0-0      0-0           NaN
94        A-40       0-0      0-0           NaN
100      40-40       0-0      0-0           NaN
103       A-40       0-0      0-0           NaN
105      40-40       0-0      0-0           NaN
118       A-40       0-0      0-0           NaN
327       40-0       2-4      0-0           NaN
348       40-0       2-4      0-0           NaN
478      40-40       4-5      0-0           NaN
486      40-40       4-5      0-0           NaN
617       40-A       1-1      0-1           NaN


## Error Check 14:
### Find all the rows where there is isPointEnd == 1 but there is no isWinner, isErrorWideL, isErrorWideR, isErrorNet, isErrorLong


In [241]:
filtered_rows = shot_data[(shot_data['isPointEnd'] == 1) &
                          (shot_data['isWinner'] != 1) &
                          (shot_data['isErrorWideL'] != 1) &
                          (shot_data['isErrorWideR'] != 1) &
                          (shot_data['isErrorNet'] != 1) & 
                          (shot_data['isErrorLong'] != 1) &
                          (shot_data['firstServeIn'] != 0) & 
                          (shot_data['secondServeIn'] != 0)]

if filtered_rows.empty:
    print ("Check passed.")
    set_background('#90EE90')
else:
    print("Error: isPointEnd == 1 but there is no point end shot")
    print(filtered_rows[['pointScore', 'gameScore', 'setScore','isPointStart', 'isPointEnd', 'isWinner', 'isErrorWideL', 'isErrorWideR', 'isErrorNet', 'isErrorLong']])
    set_background('#FF474C')

Check passed.


# Add More Columns

### Add Column: tiebreakScore

In [270]:
 # Check if both conditions passed
if 'tiebreakScore' in shot_data.columns and not shot_data['tiebreakScore'].isnull().any():
    print("Check Passed: Point Numbers already exist")
else:
    shot_data['tiebreakScore'] = np.nan
    player2_new_name = shot_data['player2Name'].iloc[0]

    # Function to reverse pointScore
    def reverse_point_score(score):
        if '-' in score:
            parts = score.split('-')
            return '-'.join(parts[::-1])
        return score

    # Apply reverse_point_score function where serverName is 'Player2'
    shot_data.loc[shot_data['serverName'] == player2_new_name, 'pointScore'] = \
        shot_data.loc[shot_data['serverName'] == player2_new_name, 'pointScore'].apply(reverse_point_score)

    # Transfer values from pointScore to tiebreakScore where gameScore is '6-6'
    shot_data.loc[shot_data['gameScore'] == '6-6', 'tiebreakScore'] = shot_data['pointScore']

    # Set the pointScore to NaN where tiebreakScore is not NaN
    shot_data.loc[pd.notna(shot_data['tiebreakScore']), 'pointScore'] = np.nan
set_background('#90EE90')

#### Print tiebreakScore

In [273]:
shot_data.loc[shot_data['gameScore'] == '6-6', 'tiebreakScore']
set_background('#90EE90')

#### print pointScore when gameScore == '6-6'
- should be all NA

In [276]:
shot_data.loc[shot_data['gameScore'] == '6-6', 'pointScore'].head()
set_background('#90EE90')

### Add Column: returnerName

In [279]:
# Get Returner Name for Shot DF

# Extract player1Name and player2Name from the first row of shot_data
first_row_shot_df = shot_data.iloc[0]
player1_name = first_row_shot_df['player1Name']
player2_name = first_row_shot_df['player2Name']

# # Can Also Manually inpute Player 1 and 2 name
# player1_name = "Kimmi Hance"
# player2_name = "Malaika Rapolu"

def get_returner_name(server_name):
    return player2_name if server_name == player1_name else player1_name

# Add 'Returner Name' column to point_df using the function
shot_data['returnerName'] = shot_data['serverName'].apply(get_returner_name)

print(f"Player 1 = {player1_name}, Player 2 = {player2_name}")
# print(shot_data[['serverName','returnerName']])
set_background('#90EE90')

Player 1 = nan, Player 2 = nan


### Add Column: shotHitBy

In [282]:
# Add Shot Hit By for each Shot

shot_data['shotHitBy'] = shot_data.apply(lambda row: row['serverName'] if row['shotInRally'] % 2 == 1 else row['returnerName'], axis=1)
print(shot_data[['serverName', 'shotHitBy','pointNumber', 'shotInRally']].head(10))
set_background('#90EE90')

  serverName shotHitBy  pointNumber  shotInRally
0    Player1   Player1            1            1
1    Player1       NaN            1            2
2    Player1   Player1            1            3
3    Player1       NaN            1            4
4    Player1   Player1            1            5
5    Player1       NaN            1            6
6    Player1   Player1            2            1
7    Player1       NaN            2            2
8    Player1   Player1            2            3
9    Player1       NaN            2            4


### Add Column: isInsideOut, isInsideIn

In [285]:
# Add columns for isInsideOut and isInsideIn, initially set to 0
shot_data['isInsideOut'] = None
shot_data['isInsideIn'] = None

# Iterate through rows
for index, row in shot_data.iterrows():
    shotHitBy = row['shotHitBy']
    player_hand = row['player1Hand'] if shotHitBy == row['player1Name'] else row['player2Hand']
    

    
    if player_hand == "Right":
        if row['side'] == "Deuce" and row['shotFhBh'] == "Backhand" and row['shotDirection'] == "Crosscourt":
            shot_data.at[index, 'isInsideOut'] = 1
        elif row['side'] == "Ad" and row['shotFhBh'] == "Forehand" and row['shotDirection'] == "Crosscourt":
            shot_data.at[index, 'isInsideOut'] = 1
        elif row['side'] == "Deuce" and row['shotFhBh'] == "Backhand" and row['shotDirection'] == "Down the Line":
            shot_data.at[index, 'isInsideIn'] = 1
        elif row['side'] == "Ad" and row['shotFhBh'] == "Forehand" and row['shotDirection'] == "Down the Line":
            shot_data.at[index, 'isInsideIn'] = 1
    elif player_hand == "Left":
        if row['side'] == "Ad" and row['shotFhBh'] == "Backhand" and row['shotDirection'] == "Crosscourt":
            shot_data.at[index, 'isInsideOut'] = 1
        elif row['side'] == "Deuce" and row['shotFhBh'] == "Forehand" and row['shotDirection'] == "Crosscourt":
            shot_data.at[index, 'isInsideOut'] = 1
        elif row['side'] == "Ad" and row['shotFhBh'] == "Backhand" and row['shotDirection'] == "Down the Line":
            shot_data.at[index, 'isInsideIn'] = 1
        elif row['side'] == "Deuce" and row['shotFhBh'] == "Forehand" and row['shotDirection'] == "Down the Line":
            shot_data.at[index, 'isInsideIn'] = 1
set_background('#90EE90')

### Add Column: isAce

In [288]:
# Add the Ace column
shot_data['isAce'] = None

for index, row in shot_data.iterrows():
    if row['isPointEnd'] == 1:
        if row['shotInRally'] == 1: # last point is serve
            if (row['firstServeIn'] == 1 or row['secondServeIn'] == 1): # either first or second serve went in
                shot_data.at[index, 'isAce'] = 1
set_background('#90EE90')

### Add Column: isDoubleFault

In [291]:
# Add the DoubleFault column
shot_data['isDoubleFault'] = None

for index, row in shot_data.iterrows():
    if row['isPointEnd'] == 1:
        if row['shotInRally'] == 1: # last point is serve
            if (row['firstServeIn'] != 1 and row['secondServeIn'] != 1): # either first or second serve went in
                shot_data.at[index, 'isDoubleFault'] = 1
set_background('#90EE90')

### Add Column: pointWonBy, lastShotError

In [294]:
# Add the 'pointWonBy' column
shot_data['pointWonBy'] = None

# Add the 'lastShotError' column
shot_data['lastShotError'] = 0

for index, row in shot_data.iterrows():
    if row['isPointEnd'] == 1:
        if row['shotInRally'] == 1: # last point is serve
            if row['isAce'] == 1: 
                shot_data.at[index, 'pointWonBy'] = row['serverName']
            elif row['isDoubleFault'] == 1: 
                shot_data.at[index, 'pointWonBy'] = row['returnerName']

                
        elif row['shotInRally'] != 1:
            if row['isErrorWideR'] == 1 or row['isErrorWideL'] == 1 or row['isErrorNet'] == 1 or row['isErrorLong'] == 1: # if error
                shot_data.at[index, 'lastShotError'] = 1
                
                if row['shotInRally'] % 2 == 0:
                    shot_data.at[index, 'pointWonBy'] = row['serverName']
                else:
                    shot_data.at[index, 'pointWonBy'] = row['returnerName']
        
            elif row['isWinner'] == 1:
                if row['shotInRally'] % 2 == 0:
                    shot_data.at[index, 'pointWonBy'] = row['returnerName']
                else:
                    shot_data.at[index, 'pointWonBy'] = row['serverName']
set_background('#90EE90')

## Error Check 15: 
#### Output Point End with no pointWonBy


In [301]:
# Missing Point End
print(shot_data[(shot_data['isPointEnd'] == 1) & (shot_data['pointWonBy'].isnull())][['pointScore', 'gameScore','setScore', 'lastShotError', 'isWinner', 'isErrorWideR', 'isErrorWideL',
       'isErrorNet', 'isErrorLong', 'pointWonBy', 'serverName', 'shotInRally']])
set_background('#90EE90')

    pointScore gameScore setScore  lastShotError  isWinner  isErrorWideR  \
10        15-0       0-0      0-0              1       NaN           NaN   
21       30-15       0-0      0-0              1       NaN           1.0   
41       40-30       0-0      0-0              0       1.0           NaN   
48       40-40       0-0      0-0              1       NaN           NaN   
67        A-40       0-0      0-0              1       NaN           1.0   
..         ...       ...      ...            ...       ...           ...   
807      30-40       3-3      0-1              0       1.0           NaN   
831        0-0       3-5      0-1              0       1.0           NaN   
834       0-15       3-5      0-1              1       NaN           NaN   
838       0-30       3-5      0-1              0       1.0           NaN   
861      30-40       3-5      0-1              1       NaN           NaN   

     isErrorWideL  isErrorNet  isErrorLong pointWonBy serverName  shotInRally  
10     

### Backward Fill: PointWonBy

In [366]:
shot_data['pointWonBy'].bfill()
set_background('#90EE90')

## Error Check 16: 
#### Find all rows with mismatched serve in/serve zone
Got an issue where jupyter notebook just can't create sets?

In [377]:
# Find indices of all rows with firstServeIn data
first_serve_row = shot_data.index[shot_data['firstServeIn'].notnull()]
# Find indices of all rows with firstServeZone data
first_serve_placement_row = shot_data.index[shot_data['firstServeZone'].notnull()]
# Check which indices are not found in one column but are found in the other
mismatched_first_serve_rows = list(set(first_serve_row).difference(first_serve_placement_row))
# Add 2 to the indices to match numbering of Google Sheets
mismatched_first_serve_rows = [x+2 for x in mismatched_first_serve_rows]
if not mismatched_first_serve_rows:
    print("Check passed for first serves.")
    set_background('#90EE90')
else:
    print("Rows where first serve in and first serve zone are not found together: " + str(mismatched_first_serve_rows))
    set_background('#FF474C')

second_serve_row = shot_data.index[shot_data['secondServeIn'].notnull()]
second_serve_placement_row = shot_data.index[shot_data['secondServeZone'].notnull()]
mismatched_second_serve_rows = list(set(second_serve_row).difference(first_serve_placement_row))
mismatched_second_serve_rows = [x+2 for x in mismatched_second_serve_rows]
if not mismatched_second_serve_rows:
    print("Check passed for second serves.")
    set_background('#90EE90')
else:
    print("Rows where second serve in and second serve zone are not found together: " + str(mismatched_second_serve_rows))
    set_background('#FF474C')

TypeError: 'list' object is not callable

### Add Column: serveResult, serveInPlacement

In [386]:
conditions = [
    (shot_data['isPointStart'] == 1) & (shot_data['firstServeIn'] == 1),
    (shot_data['isPointStart'] == 1) & (shot_data['firstServeIn'] != 1) & (shot_data['secondServeIn'] == 1),
    (shot_data['isPointStart'] == 1) & (shot_data['firstServeIn'] != 1) & (shot_data['secondServeIn'] != 1),]

# Define the values to be assigned for each condition
values_result = ['1st Serve In', '2nd Serve In', 'Double Fault']
values_placement = [shot_data['firstServeZone'], shot_data['secondServeZone'], shot_data['secondServeZone']]

# Use numpy.select to assign values based on conditions
shot_data['serveResult'] = np.select(conditions, values_result, default='')
shot_data['serveInPlacement'] = np.select(conditions, values_placement, default='')

# Now shot_data DataFrame is updated with serveResult and serveInPlacement values where isPointStart is 1
# print(shot_data[['serveResult', 'serveInPlacement','pointScore','side']].head(60))
set_background('#90EE90')

In [388]:
shot_data.replace("", None, inplace=True)
set_background('#90EE90')

In [390]:
# shot_data.query("shotInRally == 1")['serveResult'].unique()

shot_data.loc[shot_data['serveResult'] == 'Double Fault', 'serveInPlacement'].value_counts
set_background('#90EE90')

### Add Column: Depth

In [392]:
def depth_metric(shotInRally, x, y, side) :
    
    if (x >= -157.5) & (x <= 157.5):
    
        if side == 'Near':
            if shotInRally % 2 == 0:
                if -455 < y < -350: return 'Deep'
                if -350 < y < 0: return 'Short'
                if y < -455: return 'Long'

            elif shotInRally % 2 == 1:
                if 455 > y > 350: return 'Deep'
                if 0 < y < 350: return 'Short'
                if y > 455: return 'Long'

        elif side == 'Far':
            if shotInRally % 2 == 1:
                if -455 < y < -350: return 'Deep'
                if -350 < y < 0: return 'Short'
                if y < -455: return 'Long'

            elif shotInRally % 2 == 0:
                if 455 > y > 350: return 'Deep'
                if 0 < y < 350: return 'Short'
                if y > 455: return 'Long'
        
shot_data['depth'] = shot_data.apply(lambda x: depth_metric(x['shotInRally'], x['shotLocationX'], x['shotLocationY'], x['serverFarNear']), axis=1)
set_background('#90EE90')

In [394]:
shot_data[['shotInRally','serverFarNear', 'shotLocationY', 'depth']]['depth'].value_counts()
set_background('#90EE90')

In [396]:
shot_data[shot_data['depth'] == 'Long']
set_background('#90EE90')

In [398]:
shot_data[shot_data['isErrorLong'] == 1][['shotLocationX', 'shotLocationY']]
set_background('#90EE90')

### Add Column: isAprroach

# Double Check BEFORE Outputting

In [ ]:
print(shot_data.head(10))

In [ ]:
print(shot_data.tail(10))

# Output ShotCSV

In [400]:
# # # Ouput Improved Shot Csv HERE
# # Assuming point_df is your DataFrame and player1Name and player2Name are the names from the first row
player1NameNoSpace = shot_data.iloc[0]['player1Name'].replace(" ", "")
player2NameNoSpace = shot_data.iloc[0]['player2Name'].replace(" ", "")

# Save DataFrame to CSV file with modified player names
shot_data.to_csv(f'Shot_Visuals_{player1NameNoSpace}_{player2NameNoSpace}.csv', index=False)
set_background('#E9FDFF')

AttributeError: 'numpy.float64' object has no attribute 'replace'

In [ ]:
shot_data['pointNumber'].unique()

# Below is for Point CSV

### Create Point DF

In [ ]:
# Creating point_df (with only 1 row for each pointNumber)
point_df = shot_data.drop_duplicates(subset='pointNumber')[['pointNumber']]
# point_df.shape

### Add Column: player1Name, player2Name
### Set Variable: first_player1Name, first_player2Name

In [ ]:
# Extract the first value of player1Name and player2Name from shot_data
first_player1Name = shot_data['player1Name'].iloc[0]
first_player2Name = shot_data['player2Name'].iloc[0]

# Fill in the first value into all rows of point_df['player1Name'] and point_df['player2Name']
point_df['player1Name'] = first_player1Name
point_df['player2Name'] = first_player2Name

### Add Column: Scores

In [ ]:
point_df['pointScore'] = shot_data.groupby('pointNumber')['pointScore'].first().values
point_df['gameScore'] = shot_data.groupby('pointNumber')['gameScore'].first().values
point_df['setScore'] = shot_data.groupby('pointNumber')['setScore'].first().values
point_df['tiebreakScore'] = shot_data.groupby('pointNumber')['tiebreakScore'].first().values

### Add Column: Side

In [ ]:
# Group shot_data by 'pointNumber' and get the first 'side' value for each group
side_values = shot_data.groupby('pointNumber')['side'].first().reset_index()

point_df['side'] = side_values['side'].values

### Add Column: serverName, returnerName

In [ ]:
# Adds Server and Returner Names and pointScore

point_df['serverName'] = shot_data.groupby('pointNumber')['serverName'].first().values
point_df['returnerName'] = shot_data.groupby('pointNumber')['returnerName'].first().values


In [ ]:
client_team_value = shot_data.loc[0, 'clientTeam']
opponent_team_value = shot_data.loc[0, 'opponentTeam']

# Adding 'clientTeam' and 'opponentTeam' columns to point_df
point_df['clientTeam'] = client_team_value
point_df['opponentTeam'] = opponent_team_value


## Warning: Will be Empty if Timestamp separate

In [ ]:
# Add Start and End times per point

for index, row in shot_data.iterrows():
    point_number = row['pointNumber']
    
    if row['isPointStart'] == 1:
        point_df.loc[point_df['pointNumber'] == point_number, 'Position'] = row['pointStartTime']
    if row['isPointEnd'] == 1:
        point_df.loc[point_df['pointNumber'] == point_number, 'pointEndPosition'] = row['pointEndTime']

# Add Duration
point_df['Duration'] = point_df['pointEndPosition'] - point_df['Position']

In [ ]:
len(point_df['Position'])

In [ ]:
len(point_df['pointEndPosition'])

### Add Column: Rally Column
- rallyCount (x amount)
- rallyCountFreq (1-4,5-8,9-12,13+)

In [ ]:
# Find the highest shotInRally for each pointNumber in shot_data
max_rally_per_point = shot_data.groupby('pointNumber')['shotInRally'].max().reset_index()
point_df['rallyCount'] = list(max_rally_per_point['shotInRally'])

# Add 'rallyCountFreq' column based on specified conditions
point_df['rallyCountFreq'] = point_df['rallyCount'].apply(lambda x: '1 - 4' if 1 <= x <= 4 else ('5 - 8' if 5 <= x <= 8 else ('9 - 12' if 9 <= x <= 12 else ('13 +' if x >= 13 else 'Error'))))


### Add Column: Serve 
- firstServeIn
- secondServeIn
- serveResult
- serveInPlacement

Part 2
- serveResult
- serveInPlacement
 
Part 3
- firstServeZone
- secondServeZone

In [ ]:
# Add firstServeIn and secondServeIn

# Add firstServeIn and secondServeIn columns
point_df['firstServeIn'] = 0
point_df['secondServeIn'] = 0

for point_number in shot_data['pointNumber'].unique():
    # Check if firstServeIn is 1 for the given pointNumber in shot_data
    if any((shot_data['pointNumber'] == point_number) & (shot_data['firstServeIn'] == 1)):
        point_df.loc[point_df['pointNumber'] == point_number, 'firstServeIn'] = 1
    
    # Check if secondServeIn is 1 for the given pointNumber in shot_data
    if any((shot_data['pointNumber'] == point_number) & (shot_data['secondServeIn'] == 1)):
        point_df.loc[point_df['pointNumber'] == point_number, 'secondServeIn'] = 1

#part 2
start_points = shot_data[shot_data['isPointStart'] == 1]

# Set values in point_df to corresponding values from start_points
point_df['serveResult'] = start_points['serveResult'].values
point_df['serveInPlacement'] = start_points['serveInPlacement'].values

# part 3
serve_zones = shot_data.loc[shot_data['shotInRally'] == 1, ['pointNumber', 'firstServeZone', 'secondServeZone', 'firstServeIn', 'secondServeIn']].drop_duplicates()
point_df['firstServeZone'] = shot_data.groupby('pointNumber')['firstServeZone'].first().values
point_df['secondServeZone'] = shot_data.groupby('pointNumber')['secondServeZone'].first().values

### Add Column: Ace

In [ ]:
point_df['isAce'] = ((point_df['rallyCount'] == 1) & ((point_df['serveResult'] != "Double Fault")))

# Display the resulting DataFrame
print(point_df[['pointNumber', 'rallyCount', 'isAce']].head(14))

### Add Column: Server Coordinate Data

In [ ]:
point_df['serverFarNear'] = shot_data.groupby('pointNumber')['serverFarNear'].first().values

# Add first serve X and Y column
point_df['firstServeXCoord'] = shot_data.groupby('pointNumber')['firstServeXCoord'].first().values
point_df['firstServeYCoord'] = shot_data.groupby('pointNumber')['firstServeYCoord'].first().values

# Add second serve X and Y column
point_df['secondServeXCoord'] = shot_data.groupby('pointNumber')['secondServeXCoord'].first().values
point_df['secondServeYCoord'] = shot_data.groupby('pointNumber')['secondServeYCoord'].first().values


point_df[['serverFarNear', 'firstServeXCoord', 'firstServeYCoord', 'secondServeXCoord', 'secondServeYCoord']]

### Add Column: Return

In [ ]:
# print(shot_data[shot_data['shotInRally'] == 2][['shotDirection','shotFhBh','pointScore','gameScore','setScore']])

In [ ]:
# Set the initial values of 'returnDirection' and 'returnHand' columns to None
point_df['returnDirection'] = None
point_df['returnFhBh'] = None

# Iterate through pointNumber in shot_data
for point_number in shot_data['pointNumber'].unique():
    # Check if shotInRally == 2 exists for the given pointNumber
    if 2 in shot_data.loc[shot_data['pointNumber'] == point_number, 'shotInRally'].values:
        # Get the information from the corresponding row
        row_with_return_info = shot_data[(shot_data['pointNumber'] == point_number) & (shot_data['shotInRally'] == 2)].iloc[0]

        # Assign values to 'returnDirection' and 'returnHand' columns
        point_df.loc[point_df['pointNumber'] == point_number, 'returnDirection'] = row_with_return_info['shotDirection']
        point_df.loc[point_df['pointNumber'] == point_number, 'returnFhBh'] = row_with_return_info['shotFhBh']

# Display the modified DataFrame
print(point_df.head(10))

### Add Column: Error Column
- errorType

In [ ]:
# Create an empty DataFrame to store the results
error_results = pd.DataFrame(columns=['errorType', 'pointNumber'])

# Iterate through entire shot_data
for index, row in shot_data.iterrows():
    pointNumber = row['pointNumber']
    point_error_value = None
    
    if row['isErrorWideR'] == 1:
        point_error_value = 'Wide Right'
    elif row['isErrorWideL'] == 1:
        point_error_value = 'Wide Left'
    elif 'isErrorNet' in row and row['isErrorNet'] == 1:
        point_error_value = 'Net'
    elif row['isErrorLong'] == 1:
        point_error_value = 'Long'
    

    # If an error is found, append the result to the error_results DataFrame
    if point_error_value is not None:
        error_results = pd.concat([error_results, pd.DataFrame({'pointNumber': [pointNumber], 'errorType': [point_error_value]})], ignore_index=True)


# Drop duplicates based on 'pointNumber'
error_results = error_results.drop_duplicates(subset=['pointNumber'])

In [ ]:
# Create a dictionary mapping 'pointNumber' to 'errorType' in error_results
error_type_mapping = dict(zip(error_results['pointNumber'], error_results['errorType']))

# Create 'errorType' column in point_df based on the mapping
point_df['errorType'] = point_df['pointNumber'].map(error_type_mapping)

point_df = point_df.replace({np.nan: None})

### Add Column: Error (Returns)

In [ ]:
def get_return_error(row):
    if row['rallyCount'] == 2:
        return row['errorType']
    else:
        return None

point_df.loc[point_df['pointNumber'] == point_number, 'serveInPlacement'] = shot_data['secondServeZone']  

# Apply the functions to create the new columns
point_df['returnError'] = point_df.apply(get_return_error, axis=1)


### Add Column: Last Shot

In [ ]:
point_df['lastShotDirection'] = None
point_df['lastShotFhBh'] = None
point_df['lastShotHitBy'] = None  
point_df['lastShotResult'] = None  

# Iterate through unique pointNumbers in shot_data
for point_number in shot_data['pointNumber'].unique():
    # Check if isPointEnd == 1 exists for the given pointNumber
    if 1 in shot_data.loc[shot_data['pointNumber'] == point_number, 'isPointEnd'].values:
        # Get the information from the corresponding row
        row_with_lastshot_info = shot_data[(shot_data['pointNumber'] == point_number) & (shot_data['isPointEnd'] == 1)].iloc[0]

        # Assign values to 'lastShotDirection' and 'lastShotFhBh' columns
        point_df.loc[point_df['pointNumber'] == point_number, 'lastShotDirection'] = row_with_lastshot_info['shotDirection']
        point_df.loc[point_df['pointNumber'] == point_number, 'lastShotFhBh'] = row_with_lastshot_info['shotFhBh']
        point_df.loc[point_df['pointNumber'] == point_number, 'lastShotHitBy'] = row_with_lastshot_info['shotHitBy']
        
        # Determine lastShotResult based on conditions
        if row_with_lastshot_info['isWinner'] == 1 and not row_with_lastshot_info['isAce']:
            point_df.loc[point_df['pointNumber'] == point_number, 'lastShotResult'] = "Winner"
        elif row_with_lastshot_info['lastShotError'] == 1:
            point_df.loc[point_df['pointNumber'] == point_number, 'lastShotResult'] = "Error"

In [ ]:
print(point_df['lastShotResult'].unique())

### Add Column: pointWonBy

In [ ]:
# Initialize variables to keep track of the state
prev_point_number = None
point_won_by_list = []

# Iterate through the DataFrame
for index, row in shot_data.iterrows():
    if row['isPointEnd'] == 1:
        # Check if pointNumber is different and consecutively increasing
        if prev_point_number is None or row['pointNumber'] == prev_point_number + 1:
            # Append pointWonBy to the list
            point_won_by_list.append(row['pointWonBy'])
            prev_point_number = row['pointNumber']
        else:
            raise ValueError("Error: Point numbers are not different or consecutively increasing.")
            break

# Add point_won_by_list as a new column to point_df
point_df['pointWonBy'] = point_won_by_list

### Add Column: isExcitingPoint

In [ ]:
point_df['isExcitingPoint'] = shot_data.groupby('pointNumber')['isExcitingPoint'].count()

## Error Check 17: 
#### player1, player2, serverName, returnerName do not have misspellings

In [ ]:
print(point_df['player1Name'].unique())
print(point_df['player2Name'].unique())
print(point_df['serverName'].unique())
print(point_df['returnerName'].unique())

if (len(point_df['serverName'].unique()) != 2):
    raise ValueError('Error: Mispelled Names')
    
if (len(point_df['returnerName'].unique()) != 2):
    raise ValueError('Error: Mispelled Names')

## Error Check 18: 
#### pointWonBy has value that is not one of the two player names
- References: first_player1Name, first_player2Name from above server info in point_df

In [ ]:
print(point_df['pointWonBy'].unique())
none_pointWonBy_df = point_df[~point_df['pointWonBy'].isin([first_player1Name, first_player2Name])]

if none_pointWonBy_df.empty:
    print("Check passed.")
else:
    print("Error: Rows where pointWonBy is not by either player!")
    print(none_pointWonBy_df)

### Add Column: Break Point

In [ ]:
break_point_values = ['0-40', '15-40', '30-40', '40-40']

# Create 'isBreakPoint' column in point_df
point_df['isBreakPoint'] = point_df['pointScore'].isin(break_point_values)

In [ ]:
break_point_total = point_df['isBreakPoint'].sum()
print("Total break points:", break_point_total)
point_df[point_df['isBreakPoint'] == 1].head()

## Warning: Do not use if atNetPlayer1 is in shot_data
### Add Column: atNetPlayer1, atNetPlayer2

- if below says atNetPlayer1: Uncomment below Code
- replace later with atNetPlayer1 atNetPlayer2

In [ ]:
# Check if 'atNetPlayer1' exists in point_df
if 'atNetPlayer1' in shot_data.columns:
    print("Column 'atNetPlayer1' exists in point_df.")
else:
    raise ValueError("Column 'atNetPlayer1' does not exist in point_df.")


if 'atNetPlayer2' in shot_data.columns:
    print("Column 'atNetPlayer2' exists in point_df.")
else:
    raise ValueError("Column 'atNetPlayer2' does not exist in point_df.")

In [ ]:
# Add atNetPlayer1 and atNetPlayer2 columns
# point_df['atNetPlayer1'] = 0
# point_df['atNetPlayer2'] = 0

for point_number in shot_data['pointNumber'].unique():
    # Check if firstServeIn is 1 for the given pointNumber in shot_data
    if any((shot_data['pointNumber'] == point_number) & (shot_data['atNetPlayer1'] == 1)):
        point_df.loc[point_df['pointNumber'] == point_number, 'atNetPlayer1'] = 1

    # Check if secondServeIn is 1 for the given pointNumber in shot_data
    if any((shot_data['pointNumber'] == point_number) & (shot_data['atNetPlayer2'] == 1)):
        point_df.loc[point_df['pointNumber'] == point_number, 'atNetPlayer2'] = 1

### Add Column: setNum

In [ ]:
# Extract numbers from 'setScore' and calculate the sum plus 1
point_df['setNum'] = point_df['setScore'].apply(lambda x: sum(int(char) for char in x if char.isdigit()) + 1)

In [ ]:
point_df.columns

### Add Column: Game Number, Set Number, Game/Set/Point for each player

In [ ]:
point_df[['player1SetScore', 'player2SetScore']] = point_df['setScore'].str.split('-', expand=True)
point_df[['player1GameScore', 'player2GameScore']] = point_df['gameScore'].str.split('-', expand=True)
point_df[['player1PointScore', 'player2PointScore']] = point_df['pointScore'].str.split('-', expand=True)
if not point_df['tiebreakScore'].isnull().all() and not point_df['tiebreakScore'].eq("").all():
    # Perform the operation only when tiebreakScore is not empty
    point_df[['player1TiebreakScore', 'player2TiebreakScore']] = point_df['tiebreakScore'].str.split('-', expand=True)
else:
    # Set player1TiebreakScore and player2TiebreakScore to NaN
    point_df['player1TiebreakScore'] = np.nan
    point_df['player2TiebreakScore'] = np.nan
    
def calculate_game_number(score):
    return int(score.split('-')[0]) + int(score.split('-')[1])+1

# Apply the function to create the 'gameNumber' column
point_df['gameNumber'] = point_df['gameScore'].apply(calculate_game_number)

## Error Check 19: 
#### Output Game Number in order: Should be consecutive increasing. Ex: 1,2,3,4,5,6. End of Set 1. 1,2,3,4,5,6,7,8

In [ ]:
game_numbers = point_df['gameNumber'].tolist()

# Initialize variables
seen = set()
prev = None

# Iterate through gameNumber column
for num in game_numbers:
    # If the number is not in seen or it's different from the previous one, print it
    if num not in seen or num != prev:
        print(num, end=', ')
    # If the number is the same as the previous one but not consecutive, print it
    elif num == prev and num not in seen:
        print(num, end=', ')
    # Update seen set and prev variable
    seen.add(num)
    prev = num

### Add Column: player1ServeResult

In [ ]:
# Add the 'player1ServeResult' column
point_df['player1ServeResult'] = None

# Set player1ServeResult based on conditions
point_df.loc[point_df['serverName'] == point_df['player1Name'], 'player1ServeResult'] = point_df['serveResult']

In [ ]:
point_df.loc[point_df['isAce'] == True, 'player1ServeResult'] = 'Ace'


### Add Column: player1ServePlacement

In [ ]:
# Add the 'player1ServePlacement' column
point_df['player1ServePlacement'] = None

# Set player1ServePlacement based on conditions
point_df.loc[point_df['serverName'] == point_df['player1Name'], 'player1ServePlacement'] = point_df['side'] + ': ' + point_df['serveInPlacement']

### Add Column: player1ReturnPlacement

In [ ]:
# Add the 'player1ReturnPlacement' column
point_df['player1ReturnPlacement'] = None

# Set player1ServePlacement based on conditions
point_df.loc[point_df['returnerName'] == point_df['player1Name'], 'player1ReturnPlacement'] = point_df['returnDirection']

### Add Column: player1ReturnFhBh

In [ ]:
# Add the 'player1ReturnFhBh' column
point_df['player1ReturnFhBh'] = None

# Set player1ServePlacement based on conditions
point_df.loc[point_df['returnerName'] == point_df['player1Name'], 'player1ReturnFhBh'] = point_df['returnFhBh']

### Add Column: player1LastShotPlacement

In [ ]:
# Add the 'player1LastShotFhBh' column
point_df['player1LastShotPlacement'] = None

# Set player1ServePlacement based on conditions
point_df.loc[point_df['lastShotHitBy'] == point_df['player1Name'], 'player1LastShotPlacement'] = point_df['lastShotDirection']

### Add Column: player1LastShotFhBh

In [ ]:
# Add the 'player1LastShotFhBh' column
point_df['player1LastShotFhBh'] = None

# Set player1ServePlacement based on conditions
point_df.loc[point_df['lastShotHitBy'] == point_df['player1Name'], 'player1LastShotFhBh'] = point_df['lastShotFhBh']

### Add Column: player1LastShotResult

In [ ]:
# Add the 'player1LastShotResult' column
point_df['player1LastShotResult'] = None

# Set player1LastShotResult based on conditions, excluding 'Ace' and 'Double Fault'
point_df.loc[
    (point_df['lastShotHitBy'] == point_df['player1Name']) & 
    ~point_df['player1ServeResult'].isin(['Ace', 'Double Fault']), 
    'player1LastShotResult'
] = point_df['lastShotResult']


In [ ]:
# Add the 'player2ServeResult' column
point_df['player2ServeResult'] = None

# Set player1ServeResult based on conditions
point_df.loc[point_df['serverName'] == point_df['player2Name'], 'player2ServeResult'] = point_df['serveResult']

In [ ]:
point_df.loc[point_df['isAce'] == True, 'player1ServeResult'] = 'Ace'


### Add Column: player2ServePlacement

In [ ]:
# Add the 'player1ServePlacement' column
point_df['player2ServePlacement'] = None

# Set player1ServePlacement based on conditions
point_df.loc[point_df['serverName'] == point_df['player2Name'], 'player2ServePlacement'] = point_df['side'] + ': ' + point_df['serveInPlacement']

### Add Column: player2ReturnPlacement

In [ ]:
# Add the 'player2ReturnPlacement' column
point_df['player2ReturnPlacement'] = None

# Set player1ServePlacement based on conditions
point_df.loc[point_df['returnerName'] == point_df['player2Name'], 'player2ReturnPlacement'] = point_df['returnDirection']

### Add Column: player2ReturnFhBh

In [ ]:
# Add the 'player1ReturnFhBh' column
point_df['player2ReturnFhBh'] = None

# Set player1ServePlacement based on conditions
point_df.loc[point_df['returnerName'] == point_df['player2Name'], 'player2ReturnFhBh'] = point_df['returnFhBh']

### Add Column: player2LastShotPlacement

In [ ]:
# Add the 'player1LastShotFhBh' column
point_df['player2LastShotPlacement'] = None

# Set player1ServePlacement based on conditions
point_df.loc[point_df['lastShotHitBy'] == point_df['player2Name'], 'player2LastShotPlacement'] = point_df['lastShotDirection']

### Add Column: player2LastShotFhBh

In [ ]:
# Add the 'player1LastShotFhBh' column
point_df['player2LastShotFhBh'] = None

# Set player1ServePlacement based on conditions
point_df.loc[point_df['lastShotHitBy'] == point_df['player2Name'], 'player2LastShotFhBh'] = point_df['lastShotFhBh']

### Add Column: player2LastShotResult

In [ ]:
# Add the 'player2LastShotResult' column
point_df['player2LastShotResult'] = None

# Set player1LastShotResult based on conditions, excluding 'Ace' and 'Double Fault'
point_df.loc[
    (point_df['lastShotHitBy'] == point_df['player2Name']) &
    ~point_df['player2ServeResult'].isin(['Ace', 'Double Fault']),
    'player2LastShotResult'
] = point_df['lastShotResult']


In [ ]:
print(point_df[['setNum', 'gameScore','pointScore','player1ServePlacement','serveInPlacement']].head(10))

In [ ]:
print(point_df[['setNum', 'gameScore','pointScore','player1ServePlacement','serveInPlacement']].head(10))

### Add Columns: Depths Count (Short, Deep)

In [ ]:
# Add counts for each player on how many short and deep balls they hit in the point (group by pointNumber)
# - don't have to do long since we already know what points end with isErrorLong (and also isErrorNet)



## Add Column: Name (For Video)

In [ ]:
# Change pointScore to the specified format
point_df['Name'] = point_df.apply(lambda row: f"Set {row['setNum']}: {row['gameScore']}, {row['tiebreakScore']} {row['serverName']} Serving" if pd.notna(row['tiebreakScore']) else f"Set {row['setNum']}: {row['gameScore']}, {row['pointScore']} {row['serverName']} Serving", axis=1)


In [ ]:
point_df[['setNum', 'gameScore','pointScore','tiebreakScore','Name']].head(5)

In [ ]:
point_df[['setNum', 'gameScore','pointScore','tiebreakScore','Name']].tail(5)

## Reorder DataFrame for Output

In [ ]:
point_df_copy = point_df.copy()

In [ ]:
point_df.shape

In [ ]:
point_df_copy.shape

In [ ]:
point_df.columns

In [ ]:
desired_order = [
    'Name', 'pointNumber', 'setNum', 'gameNumber', 'player1Name', 'player2Name',
    'pointScore', 'gameScore', 'setScore', 'tiebreakScore', 'side', 'serverName',
    'returnerName', 'Position', 'pointEndPosition', 'Duration', 'pointWonBy',
    'rallyCount', 'rallyCountFreq', 'firstServeIn', 'secondServeIn',
    'serveResult', 'serveInPlacement', 'firstServeZone', 'secondServeZone',
    'isAce','serverFarNear', 'firstServeXCoord', 'firstServeYCoord', 'secondServeXCoord', 'secondServeYCoord','returnDirection', 'returnFhBh', 'errorType', 'returnError',
    'lastShotDirection', 'lastShotFhBh', 'lastShotHitBy', 'lastShotResult',
    'isBreakPoint', 'atNetPlayer1', 'atNetPlayer2', 'isExcitingPoint', 'player1SetScore',
    'player2SetScore', 'player1GameScore', 'player2GameScore', 'player1PointScore',
    'player2PointScore', 'player1TiebreakScore', 'player2TiebreakScore',
    'player1ServeResult', 'player1ServePlacement', 'player1ReturnPlacement',
    'player1ReturnFhBh', 'player1LastShotPlacement', 'player1LastShotFhBh',
    'player1LastShotResult', 'player2ServeResult', 'player2ServePlacement',
       'player2ReturnPlacement', 'player2ReturnFhBh',
       'player2LastShotPlacement', 'player2LastShotFhBh',
       'player2LastShotResult', 'clientTeam', 'opponentTeam'
]

# Reorder the columns
point_df = point_df.reindex(columns=desired_order)

## Error Check 20: 
#### Check if the columns and their order are the same

In [ ]:
print(point_df.shape)
print(point_df_copy.shape)

if (point_df.shape == point_df_copy.shape):
    print('Check passed.')
else:
    raise ValueError('Error: Not the same!')

In [ ]:
# Get the set of column names for each DataFrame
point_df_columns = set(point_df.columns)
point_df_copy_columns = set(point_df_copy.columns)

# Find the column names unique to each DataFrame
unique_to_point_df = point_df_columns - point_df_copy_columns
unique_to_point_df_copy = point_df_copy_columns - point_df_columns

# Output the results
if unique_to_point_df:
    print("Columns unique to point_df:", unique_to_point_df)
else:
    print("All columns in point_df are also in point_df_copy")

if unique_to_point_df_copy:
    print("Columns unique to point_df_copy:", unique_to_point_df_copy)
else:
    print("All columns in point_df_copy are also in point_df")


## Warning: All nan to ""
### Alterating Data

In [ ]:
# Change Na to "" Empty String

# Assuming point_df is your DataFrame
point_df.replace([pd.NA, None, pd.NaT, float('nan')], "", inplace=True)

# Double Check before Outputting Point (FOR VISUAL)

In [ ]:
print(point_df.head(5))

In [ ]:
print(point_df.tail(5))

In [ ]:
# Change Value Names for Match Viewer Output

# Assuming point_df is your DataFrame and player1Name is the name from the first row
player1Name = point_df.iloc[0]['player1Name']
player2Name = point_df.iloc[0]['player2Name']

# Replace values in the 'atNetPlayer1' column
point_df['atNetPlayer1'] = point_df['atNetPlayer1'].replace({0: "", 1: player1Name})
point_df['atNetPlayer2'] = point_df['atNetPlayer2'].replace({0: "", 1: player2Name})

# PRINT POINTCSV VISUALS

In [ ]:
# Save point_df to CSV file

# Assuming point_df is your DataFrame and player1Name and player2Name are the names from the first row
player1NameNoSpace = point_df.iloc[0]['player1Name'].replace(" ", "")
player2NameNoSpace = point_df.iloc[0]['player2Name'].replace(" ", "")

# Save DataFrame to CSV file with modified player names
point_df.to_csv(f'Point_Visuals_{player1NameNoSpace}_{player2NameNoSpace}.csv', index=False)


In [ ]:
def csv_to_json(csv_file_path, json_file_name):
    # Load CSV file into a pandas DataFrame
    df = pd.read_csv(csv_file_path)
    
    # Convert DataFrame to JSON format
    json_data = df.to_json(orient='records')
    
    # Save JSON data to a file in the same directory
    json_file_path = f'{json_file_name}'
    with open(json_file_path, 'w') as json_file:
        json_file.write(json_data)
    
    return json_file_path

# Define the CSV file path
csv_file_path = f"Point_Visuals_{player1NameNoSpace}_{player2NameNoSpace}.csv"

# Define the desired JSON file name (without extension)
json_file_name = f"Point_Visuals_{player1NameNoSpace}_{player2NameNoSpace}.json"

# Convert CSV to JSON and save in the same directory
output_file_path = csv_to_json(csv_file_path, json_file_name)

# Display the path where the JSON file is saved
output_file_path


# EDA

## Shot CSV EDA

In [ ]:
# Can input CSV Directly here for statistics functions

# your_file_name = "filename.csv"
# shot_eda = pd.read_csv(your_file_name)

# if directly from notebook
shot_eda = shot_data.copy()

In [ ]:
player1Name = shot_eda.iloc[0]['player1Name']

# Filter shot_data based on the conditions
approach_data_player1 = shot_eda[(shot_eda['isApproach'] == 1) & (shot_eda['shotHitBy'] == player1Name)]

# Count the distinct pointNumbers
distinct_point_numbers = approach_data_player1['pointNumber'].nunique()

# Print the result
print(f"Number of Approach Shots hit by {player1Name}: {distinct_point_numbers}" )

# print(approach_data_player1)


## Point CSV EDA

In [ ]:
# # Can input CSV Directly here for statistics functions
# import pandas as pd
# your_file_name = "Point_Visuals_Anne-ChristineLutkemeyer_SehaYu.csv"
# point_df_eda = pd.read_csv(your_file_name)

# # OR MAKE point_df_eda copy of point_df

point_df_eda = point_df.copy()

In [ ]:
print(point_df_eda.columns)

In [ ]:
serve_in_rows = point_df_eda[(point_df_eda['serveResult'] == '1st Serve In') | (point_df_eda['serveResult'] == '2nd Serve In')]

# filter the rows where rallyCount column is equal to 1
desired_rows = serve_in_rows[serve_in_rows['rallyCount'] == 1]

# print each row
for index, row in desired_rows.iterrows():
    print(row[['pointNumber', 'setNum', 'gameNumber','serveResult','rallyCount' ,'serverName']])


In [ ]:
first_player1Name = point_df_eda['player1Name'].iloc[0]



# Display the results
print(f"\nServe Results for {first_player1Name}:")

# Assuming point_df_eda is your DataFrame
total_serves = len(point_df_eda[point_df_eda['serverName'] == first_player1Name])
first_serve_in_count = len(point_df_eda[(point_df_eda['serverName'] == first_player1Name) & (point_df_eda['firstServeIn'] == 1)])
first_serve_won_count = len(point_df_eda[(point_df_eda['serverName'] == first_player1Name) & (point_df_eda['firstServeIn'] == 1) & (point_df_eda['pointWonBy'] == first_player1Name)])
percentage_first_serve_in = (first_serve_in_count / total_serves) * 100 if total_serves > 0 else 0
percentage_first_serve_won = (first_serve_won_count / first_serve_in_count) * 100 if first_serve_in_count > 0 else 0

second_serve_total_count = len(point_df_eda[(point_df_eda['serverName'] == first_player1Name) & (point_df_eda['firstServeIn'] == 0)])
second_serve_in_count = len(point_df_eda[(point_df_eda['serverName'] == first_player1Name) & (point_df_eda['firstServeIn'] == 0)& (point_df_eda['secondServeIn'] == 1)])
second_serve_won_count = len(point_df_eda[(point_df_eda['serverName'] == first_player1Name) & (point_df_eda['firstServeIn'] == 0)& (point_df_eda['secondServeIn'] == 1) & (point_df_eda['pointWonBy'] == first_player1Name)])
percentage_second_serve_in = (second_serve_in_count / second_serve_total_count) * 100 if second_serve_total_count > 0 else 0
percentage_second_serve_won = (second_serve_won_count / second_serve_in_count) * 100 if second_serve_in_count > 0 else 0



# Display the results
print("\nTotal Serves:", total_serves)
print("First Serve In (Count):", first_serve_in_count)
print("First Serve Won (Count):", first_serve_won_count)
print(f"First Serve In (%): {percentage_first_serve_in:.2f}%")
print(f"First Serve Won (%): {percentage_first_serve_won:.2f}%")

print("Second Serve In (Count):", second_serve_in_count)
print("Second Serve Total (Count):", second_serve_total_count)
print("Second Serve Won (Count):", second_serve_won_count)
print(f"Second Serve In (%): {percentage_second_serve_in:.2f}%")
print(f"Second Serve Won (%): {percentage_second_serve_won:.2f}%")

# Assuming point_df is your DataFrame
count_is_ace = (point_df_eda[point_df_eda['serverName'] == first_player1Name]['isAce']).sum()
count_is_double_fault = ((point_df_eda['serverName'] == first_player1Name) & (point_df_eda['serveResult'] == "Double Fault")).sum()

# Display the results
print("Ace (Count):", count_is_ace)
print("Double Fault (Count):", count_is_double_fault)

# Count of rows where serverName is equal to the first row of player1Name and pointWonBy is equal to the first row of player1Name
total_service_points_won = len(point_df_eda[(point_df_eda['serverName'] == first_player1Name) & (point_df_eda['pointWonBy'] == first_player1Name)])
total_service_points_won_percentage = total_service_points_won / total_serves *100

# Display the results
print(f"Points Won on Serve (Count) {total_service_points_won}")

print(f"Points Won on Serve (%): {total_service_points_won_percentage:.2f}%")

# Assuming point_df is your DataFrame
return_points = point_df_eda[(point_df_eda['returnerName'] == first_player1Name)] # CHANGED THIS JERRY, REMOVED RALLY COUNT >= 2

total_return = len(return_points)
returnMade = len(return_points[(return_points['rallyCount'] > 2) | ((return_points['rallyCount'] == 2) & (return_points['lastShotResult'] != 'Error'))])
returnError = len(return_points[(return_points['lastShotResult'] == 'Error') & (return_points['rallyCount'] == 2)])
returnWinner = len(return_points[(return_points['lastShotResult'] == 'Winner') & (return_points['rallyCount'] == 2)])
returnMadePercentage = returnMade/total_return

returnWonByPlayer1 = len(return_points[return_points['pointWonBy'] == first_player1Name])
returnWonByPlayer1Percentage = returnWonByPlayer1 / returnMade * 100 if returnMade > 0 else 0

deuceReturnCount = len(return_points[return_points['side'] == 'Deuce'])
adReturnCount = len(return_points[return_points['side'] == 'Ad'])


deuceReturnMade = len(return_points[(return_points['side'] == 'Deuce') & ((return_points['rallyCount'] > 2) | ((return_points['rallyCount'] == 2) & (return_points['lastShotResult'] != 'Error')))])
adReturnMade = len(return_points[(return_points['side'] == 'Ad') & ((return_points['rallyCount'] > 2) | ((return_points['rallyCount'] == 2) & (return_points['lastShotResult'] != 'Error')))])

deuceReturnMadePercentage = deuceReturnMade/deuceReturnCount
adReturnMadePercentage = adReturnMade/adReturnCount

deuceReturnWonByPlayer1 = len(return_points[(return_points['side'] == 'Deuce') & (return_points['pointWonBy'] == first_player1Name) | ((return_points['rallyCount'] == 2) & (return_points['lastShotResult'] != 'Error'))])
adReturnWonByPlayer1 = len(return_points[(return_points['side'] == 'Ad') & (return_points['pointWonBy'] == first_player1Name) | ((return_points['rallyCount'] == 2) & (return_points['lastShotResult'] != 'Error'))])

deuceReturnWonByPlayer1Percentage = deuceReturnWonByPlayer1 / deuceReturnMade * 100 if deuceReturnMade > 0 else 0
adReturnWonByPlayer1Percentage = adReturnWonByPlayer1 / adReturnMade * 100 if adReturnMade > 0 else 0




print(f"\nReturn Results for {first_player1Name}:\n")

print("Total Return (Count):", total_return)
print("Return Won (Count):", returnWonByPlayer1)
print("Return Won (%):", returnWonByPlayer1Percentage)

print("\nReturn Made (Count):", returnMade)
print("Return Made (%):", returnMadePercentage)
print("Return Error (Count):", returnError)
print("Return Winner (Count):", returnWinner)

print("\nDeuce Return (Count):", deuceReturnCount)
print("Deuce Return Made (Count):", deuceReturnMade)
print("Deuce Return Made (%):", deuceReturnMadePercentage)
print("Deuce Return Won by Player1 (%):", deuceReturnWonByPlayer1Percentage)
print("Deuce Return Won by Player1 (Count):", deuceReturnWonByPlayer1)


print("\nAd Return (Count):", adReturnCount)
print("Ad Return Made (Count):", adReturnMade)
print("Ad Return Made (%):", adReturnMadePercentage)
print("Ad Return Won by Player1 (Count):", adReturnWonByPlayer1)
print("Ad Return Won by Player1 (%):", adReturnWonByPlayer1Percentage)

# Assuming return_points is your DataFrame
deuce_return_points = return_points[(return_points['side'] == 'Deuce') & (return_points['returnerName'] == first_player1Name) & (return_points['rallyCount'] >= 2)]

# Deuce Return Points Separated by returnFhBh
deuce_forehand_return_points = deuce_return_points[deuce_return_points['returnFhBh'] == 'Forehand']
deuce_backhand_return_points = deuce_return_points[deuce_return_points['returnFhBh'] == 'Backhand']


# Count for Deuce Return Points - Made
count_deuce_forehand_made = len(deuce_forehand_return_points[(deuce_forehand_return_points['rallyCount'] > 2) | ((deuce_forehand_return_points['rallyCount'] == 2) & (deuce_forehand_return_points['lastShotResult'] != 'Error'))])
count_deuce_backhand_made = len(deuce_backhand_return_points[(deuce_backhand_return_points['rallyCount'] > 2) | ((deuce_backhand_return_points['rallyCount'] == 2) & (deuce_backhand_return_points['lastShotResult'] != 'Error'))])

# Count for Deuce Return Points - Error
count_deuce_forehand_error = len(deuce_forehand_return_points[(deuce_forehand_return_points['lastShotResult'] == 'Error') & (deuce_forehand_return_points['rallyCount'] == 2)])
count_deuce_backhand_error = len(deuce_backhand_return_points[(deuce_backhand_return_points['lastShotResult'] == 'Error') & (deuce_backhand_return_points['rallyCount'] == 2)])

# Display the counts
print("\nDeuce Forehand Return Points - Made:", count_deuce_forehand_made)
print("Deuce Forehand Return Points - Error:", count_deuce_forehand_error)

print("Deuce Backhand Return Points - Made:", count_deuce_backhand_made)
print("Deuce Backhand Return Points - Error:", count_deuce_backhand_error)

# Assuming return_points is your DataFrame
ad_return_points = return_points[(return_points['side'] == 'Ad') & (return_points['returnerName'] == first_player1Name) & (return_points['rallyCount'] >= 2)]

# Ad Return Points Separated by returnFhBh
ad_forehand_return_points = ad_return_points[ad_return_points['returnFhBh'] == 'Forehand']
ad_backhand_return_points = ad_return_points[ad_return_points['returnFhBh'] == 'Backhand']

# Count for Ad Return Points - Made
count_ad_forehand_made = len(ad_forehand_return_points[(ad_forehand_return_points['rallyCount'] > 2) | ((ad_forehand_return_points['rallyCount'] == 2) & (ad_forehand_return_points['lastShotResult'] != 'Error'))])
count_ad_backhand_made = len(ad_backhand_return_points[(ad_backhand_return_points['rallyCount'] > 2) | ((ad_backhand_return_points['rallyCount'] == 2) & (ad_backhand_return_points['lastShotResult'] != 'Error'))])

# Count for Ad Return Points - Error
count_ad_forehand_error = len(ad_forehand_return_points[(ad_forehand_return_points['lastShotResult'] == 'Error') & (ad_forehand_return_points['rallyCount'] == 2)])
count_ad_backhand_error = len(ad_backhand_return_points[(ad_backhand_return_points['lastShotResult'] == 'Error') & (ad_backhand_return_points['rallyCount'] == 2)])

# Display the counts
print("\nAd Forehand Return Points - Made:", count_ad_forehand_made)
print("Ad Forehand Return Points - Error:", count_ad_forehand_error)

print("Ad Backhand Return Points - Made:", count_ad_backhand_made)
print("Ad Backhand Return Points - Error:", count_ad_backhand_error)

print(f"\nAt Net Results for {first_player1Name}:\n")


# Total points where atNetPlayer1 = first_player1Name
total_at_net_player1 = len(point_df_eda[point_df_eda['atNetPlayer1'] == first_player1Name])

# Percentage of points where atNetPlayer1 = 1 out of total points
percentage_at_net_player1 = (total_at_net_player1 / len(point_df_eda)) * 100 if len(point_df_eda) > 0 else 0

# Display the total count and percentage of points where atNetPlayer1 = 1
print(f"Total Net Points for {first_player1Name}: {total_at_net_player1}")
print(f"Percentage of Net Points for {first_player1Name}: {percentage_at_net_player1:.2f}%")

# Points where atNetPlayer1 = first_player1Name and pointWonBy = first_player1Name
at_net_player1_and_won_by_player1 = len(point_df_eda[(point_df_eda['atNetPlayer1'] == first_player1Name) & (point_df_eda['pointWonBy'] == first_player1Name)])

# Percentage of points where atNetPlayer1 = first_player1Name and pointWonBy = first_player1Name out of total points where atNetPlayer1 = 1
percentage_at_net_player1_and_won_by_player1 = (at_net_player1_and_won_by_player1 / total_at_net_player1) * 100 if total_at_net_player1 > 0 else 0

# Display the count and percentage of points where atNetPlayer1 = 1 and pointWonBy = first_player1Name
print(f"\nTotal Net Points won by {first_player1Name}: {at_net_player1_and_won_by_player1}")
print(f"Percentage of Net Points won by {first_player1Name}: {percentage_at_net_player1_and_won_by_player1:.2f}%")


In [ ]:
# Caitlin Breakpoint Data

# points_returned = point_df_eda[point_df_eda[]]

# print(f"\nBreakpoint Results for {first_player1Name}:\n")

# # Total points where isBreakPoint = 1
# total_breakpoint = len(point_df_eda[point_df_eda['isBreakPoint'] == 1])

# # Points where isBreakPoint = 1 and pointWonBy = first_player1Name
# breakpoint_and_won_by_player1 = len(point_df_eda[(point_df_eda['isBreakPoint'] == 1) & (point_df_eda['pointWonBy'] == first_player1Name)])

# # Percentage of points where isBreakPoint = 1 and pointWonBy = first_player1Name out of total points where isBreakPoint = 1
# percentage_breakpoint_and_won_by_player1 = (breakpoint_and_won_by_player1 / total_breakpoint) * 100 if total_breakpoint > 0 else 0

# # Display the total count of points where isBreakPoint = 1
# print(f"Total Breakpoints: {total_breakpoint}")

# # Display the count and percentage of points where isBreakPoint = 1 and pointWonBy = first_player1Name
# print(f"Total Breakpoints won by {first_player1Name}: {breakpoint_and_won_by_player1}")
# print(f"Percentage of Breakpoints won by {first_player1Name}: {percentage_breakpoint_and_won_by_player1:.2f}%")

# # Total points where isBreakPoint = 1 and serverName = first_player1Name
# total_breakpoint_serve = len(point_df_eda[(point_df_eda['isBreakPoint'] == 1) & (point_df_eda['serverName'] == first_player1Name)])

# # Points where isBreakPoint = 1, serverName = first_player1Name, and pointWonBy = first_player1Name
# breakpoint_and_won_by_player1_serve = len(point_df_eda[(point_df_eda['isBreakPoint'] == 1) & (point_df_eda['serverName'] == first_player1Name) & (point_df_eda['pointWonBy'] == first_player1Name)])

# # Percentage of points where isBreakPoint = 1, serverName = first_player1Name, and pointWonBy = first_player1Name out of total points where isBreakPoint = 1 and serverName = first_player1Name
# percentage_breakpoint_and_won_by_player1_serve = (breakpoint_and_won_by_player1_serve / total_breakpoint_serve) * 100 if total_breakpoint_serve > 0 else 0

# # Display the total count of points where isBreakPoint = 1 and serverName = first_player1Name
# print(f"\nTotal Breakpoints on Serve for {first_player1Name}: {total_breakpoint_serve}")

# # Display the count and percentage of points where isBreakPoint = 1, serverName = first_player1Name, and pointWonBy = first_player1Name
# print(f"Total Breakpoints won on Serve by {first_player1Name}: {breakpoint_and_won_by_player1_serve}")
# print(f"Percentage of Breakpoints won on Serve by {first_player1Name}: {percentage_breakpoint_and_won_by_player1_serve:.2f}%")

# Total points where isBreakPoint = 1 and returnerName = first_player1Name
total_breakpoint_return = len(point_df_eda[(point_df_eda['isBreakPoint'] == 1) & (point_df_eda['returnerName'] == first_player1Name)])

# Points where isBreakPoint = 1, returnerName = first_player1Name, and pointWonBy = first_player1Name
breakpoint_and_won_by_player1_return = len(point_df_eda[(point_df_eda['isBreakPoint'] == 1) & (point_df_eda['returnerName'] == first_player1Name) & (point_df_eda['pointWonBy'] == first_player1Name)])

# Percentage of points where isBreakPoint = 1, returnerName = first_player1Name, and pointWonBy = first_player1Name out of total points where isBreakPoint = 1 and returnerName = first_player1Name
percentage_breakpoint_and_won_by_player1_return = (breakpoint_and_won_by_player1_return / total_breakpoint_return) * 100 if total_breakpoint_return > 0 else 0

# Display the total count of points where isBreakPoint = 1 and returnerName = first_player1Name
print(f"\nTotal Breakpoints on Return for {first_player1Name}: {total_breakpoint_return}")

# Display the count and percentage of points where isBreakPoint = 1, returnerName = first_player1Name, and pointWonBy = first_player1Name
print(f"Total Breakpoints won on Return by {first_player1Name}: {breakpoint_and_won_by_player1_return}")
print(f"Percentage of Breakpoints won on Return by {first_player1Name}: {percentage_breakpoint_and_won_by_player1_return:.2f}%")

# Jimmy Returning Games Won
# games won/returning games by Jimmy Hou

points_returned = point_df_eda[point_df_eda["returnerName"] == first_player1Name]

# # Return percentage won on first serve
# first_serves_won = points_returned[(points_returned['firstServeIn'] == 1) & (points_returned['pointWonBy'] == first_player1Name)]
# total_first_serves = points_returned[points_returned['firstServeIn'] == 1]
# fs_won_per_player1 = 100 * len(first_serves_won) / len(total_first_serves)
# # print(f"\n{player1} won {fs_won_per_player1:.2f}% of first serves returned.")
# print(f"\nPerecentage of Breakpoinnts won by {first_player1Name} on Return when returning a first serve: {fs_won_per_player1:.2f}%")

# # Return percentage won on second serve
# second_serves_won = points_returned[(points_returned['secondServeIn'] == 1) & (points_returned['pointWonBy'] == first_player1Name)]
# total_second_serves = points_returned[points_returned['secondServeIn'] == 1]
# ss_won_per_player1 = 100 * len(second_serves_won) / len(total_second_serves)
# # print(f"{player1} won {ss_won_per_player1:.2f}% of second serves returned."
# print(f"Perecentage of Breakpoinnts won by {first_player1Name} on Return when returning a second serve: {ss_won_per_player1:.2f}%")


In [ ]:
point_df_eda.columns

In [ ]:
# Filter points where serverName is equal to first_player1Name
filtered_points = point_df[point_df['serverName'] == first_player1Name]

# Group the filtered points by player1ServePlacement and count the occurrences
serve_placement_counts = filtered_points.groupby('player1ServePlacement').size()

serve_placements_ad = serve_placement_counts.filter(like='Ad').sum()
serve_placements_deuce = serve_placement_counts.filter(like='Deuce').sum()

# Iterate over filtered_points
for index, point in filtered_points.iterrows():
    serve_placement = point['player1ServePlacement']

    # Check if serve placement is not in serve_placement_counts
    if serve_placement not in serve_placement_counts:
        print(point)


# Initialize dictionaries to store counts and percentages
point_won_counts = {}
point_won_percentages = {}
print(f"Total {len(filtered_points)}")

# Iterate over serve placements
for serve_placement, count in serve_placement_counts.items():
    # Filter points with the specific serve placement
    serve_placement_points = filtered_points[filtered_points['player1ServePlacement'] == serve_placement]
    
    # Count points won by first_player1Name
    point_won_count = serve_placement_points[serve_placement_points['pointWonBy'] == first_player1Name].shape[0]

    # Calculate percentage
    point_won_percentage = (point_won_count / count) * 100 if count > 0 else 0

    # Store counts and percentages
    point_won_counts[serve_placement] = point_won_count
    point_won_percentages[serve_placement] = point_won_percentage
    


    
    
# Filter serve placements for Ad and Deuce [CHANGED: BRIAN NTOES]
serve_placements_ad = serve_placement_counts.filter(like='Ad').sum()
serve_placements_deuce = serve_placement_counts.filter(like='Deuce').sum()    

# Print counts and percentages [CHANGED: BRIAN NOTES]
for serve_placement, count in serve_placement_counts.items():
    print(f"Serve Placement: {serve_placement}")
    print(f"Total Serves: {count}")
    
    if "Deuce" in serve_placement: # [CHANGED: BRIAN NOTES]
        deuce_serve_format_number = f"{count/serve_placements_deuce:.2f}"
        deuce_serve_percent = round(float(deuce_serve_format_number) * 100,2)
        print(f"Serve Frequency: {deuce_serve_percent}% ({count}/{serve_placements_deuce}) Deuce Serves")
    if "Ad" in serve_placement: # [CHANGED: BRIAN NOTES]
        ad_serve_format_number = f"{count/serve_placements_ad:.2f}"
        ad_serve_percent = round(float(ad_serve_format_number) * 100,2)
        print(f"Serve Frequency: {ad_serve_percent}% ({count}/{serve_placements_ad}) Ad Serves")
        
    print(f"Serves Won by {first_player1Name}: {point_won_counts.get(serve_placement, 0)}")
    print(f"Percentage: {point_won_percentages.get(serve_placement, 0):.2f}%\n")

    
# print("This is Ad count: " + str(len(point_df[(point_df['serverName'] == first_player1Name) & (point_df['side'] == 'Ad')])))
# print("This is Deuce count: " + str(len(point_df[(point_df['serverName'] == first_player1Name) & (point_df['side'] == 'Deuce')])))

In [ ]:
print(f"\nError Data for {first_player1Name}:\n")
# Filter the DataFrame based on specified conditions
total_errors = point_df[(point_df['lastShotHitBy'] == first_player1Name) &
                           (point_df['lastShotResult'] == 'Error')]

import numpy as np

# Filter rows without NaN values in relevant columns
forehand_errors = point_df[(point_df['lastShotHitBy'] == first_player1Name) &
                           (point_df['lastShotResult'] == 'Error') &
                           (point_df['lastShotFhBh'] == 'Forehand') &
                           (~point_df['errorType'].isnull())]  # Ensure 'errorType' column doesn't have NaN
 
backhand_errors = point_df[(point_df['lastShotHitBy'] == first_player1Name) &
                           (point_df['lastShotResult'] == 'Error') &
                           (point_df['lastShotFhBh'] == 'Backhand') &
                           (~point_df['errorType'].isnull())]  # Ensure 'errorType' column doesn't have NaN

# Count the occurrences of 'Forehand' and 'Backhand' separately
forehand_counts = forehand_errors.shape[0]  # Count rows
backhand_counts = backhand_errors.shape[0]  # Count rows

# Print the total error counts for verification
total_error_counts = forehand_counts + backhand_counts



# Desired output order 
desired_order = ['Net', 'Long', 'Wide Right', 'Wide Left'] 

# # Get value counts of 'errorType' for Forehand errors [CHANGED: BRIAN]
# forehand_error_types = forehand_errors['errorType'].value_counts(dropna=False).loc[desired_order]  # Include NaN values in count
# forehand_error_types_df = pd.DataFrame(forehand_error_types) # change into dataframe to erase object line
# Get value counts of 'errorType' for Backhand errors



######################### CHANGED SECTION ##############################################
# [CHANGED: BRIAN]

forehand_error_types = forehand_errors['errorType'].value_counts(dropna=False)

# Create a Series with desired index containing zeros
zeros_series = pd.Series(0, index=desired_order)

# Combine the original Series with the zeros Series
forehand_error_types_combined = forehand_error_types.combine(zeros_series, max, fill_value=0)

# Reindex the Series to follow the desired order
forehand_error_types_ordered = forehand_error_types_combined.reindex(desired_order, fill_value=0)

# Create the DataFrame
forehand_error_types_df = pd.DataFrame(forehand_error_types_ordered, columns=['Count'])

# Get value counts of 'errorType' for Backhand errors [CHANGED: BRIAN]
# backhand_error_types = backhand_errors['errorType'].value_counts(dropna=False).loc[desired_order]  # Include NaN values in count
# backhand_error_types_df = pd.DataFrame(backhand_error_types) # change into dataframe to erase object line

# Get value counts of 'errorType' for Backhand errors
backhand_error_types = backhand_errors['errorType'].value_counts(dropna=False)

# Combine the original Series with the zeros Series
backhand_error_types_combined = backhand_error_types.combine(zeros_series, max, fill_value=0)

# Reindex the Series to follow the desired order
backhand_error_types_ordered = backhand_error_types_combined.reindex(desired_order, fill_value=0)

# Create the DataFrame
backhand_error_types_df = pd.DataFrame(backhand_error_types_ordered, columns=['Count'])



######################### CHANGED SECTION ##############################################


# Print the counts and error types
print("Count of Total errors:", total_error_counts)
print("Count of Forehand errors:", forehand_counts)
print(f"Forehand Error %: {(forehand_counts/total_error_counts)*100:.2f}%")
print("Count of Backhand errors:", backhand_counts)
print(f"Backhand Error %: {(backhand_counts/total_error_counts)*100:.2f}%")
print("\nForehand errors:\n", forehand_error_types_df)


# # Group by both 'lastShotDirection' and 'errorType', and then count occurrences
forehand_error_counts = forehand_errors.groupby(['player1LastShotPlacement', 'errorType']).size().unstack(fill_value=0)  # Fill NaN with 0
forehand_error_counts_ordered = forehand_error_counts.reindex(columns = desired_order) # [CHANGED: BRIAN]

print("\nValue counts of 'errorType' for Forehand errors with different directions:\n", forehand_error_counts_ordered)


print("\nBackhand errors:\n", backhand_error_types_df)


# Group by both 'lastShotDirection' and 'errorType', and then count occurrences
backhand_error_counts = backhand_errors.groupby(['player1LastShotPlacement', 'errorType']).size().unstack(fill_value=0) # Fill NaN with 0
backhand_error_counts_ordered = backhand_error_counts.reindex(columns = desired_order) # [CHANGED: BRIAN]

print("\nValue counts of 'errorType' for Backhand errors with different directions:\n", backhand_error_counts_ordered)

# CONDENSED REPORT (NEEDS UPDATE)

In [ ]:
# pip install reportlab

In [ ]:
### from reportlab.lib import colors
# from reportlab.lib.pagesizes import letter
# from reportlab.platypus import SimpleDocTemplate, Table, TableStyle

# # Your raw output data
# raw_data = [
#     {"Serve Placement": "Ad: Body", "Total Serves": 3, "Serves Won by Tian Fang Ran": 3, "Percentage": "100.00%"},
#     {"Serve Placement": "Ad: T", "Total Serves": 11, "Serves Won by Tian Fang Ran": 8, "Percentage": "72.73%"},
#     {"Serve Placement": "Ad: Wide", "Total Serves": 19, "Serves Won by Tian Fang Ran": 10, "Percentage": "52.63%"},
#     {"Serve Placement": "Deuce: Body", "Total Serves": 5, "Serves Won by Tian Fang Ran": 2, "Percentage": "40.00%"},
#     {"Serve Placement": "Deuce: T", "Total Serves": 19, "Serves Won by Tian Fang Ran": 10, "Percentage": "52.63%"},
#     {"Serve Placement": "Deuce: Wide", "Total Serves": 14, "Serves Won by Tian Fang Ran": 7, "Percentage": "50.00%"}
# ]

# # Convert raw data to list of lists for table
# table_data = [["Serve Placement", "Total Serves", "Serves Won by Tian Fang Ran", "Percentage"]]
# for row in raw_data:
#     table_data.append([row["Serve Placement"], row["Total Serves"], row["Serves Won by Tian Fang Ran"], row["Percentage"]])

# # Create PDF
# pdf_filename = "tennis_stats.pdf"
# doc = SimpleDocTemplate(pdf_filename, pagesize=letter)
# table = Table(table_data)

# # Add style to table
# style = TableStyle([('BACKGROUND', (0, 0), (-1, 0), colors.gray),
#                     ('TEXTCOLOR', (0, 0), (-1, 0), colors.whitesmoke),
#                     ('ALIGN', (0, 0), (-1, -1), 'CENTER'),
#                     ('FONTNAME', (0, 0), (-1, 0), 'Helvetica-Bold'),
#                     ('BOTTOMPADDING', (0, 0), (-1, 0), 12),
#                     ('BACKGROUND', (0, 1), (-1, -1), colors.beige),
#                     ('GRID', (0, 0), (-1, -1), 1, colors.black)])
# table.setStyle(style)

# # Add table to PDF
# doc.build([table])
